In [33]:
import os
import pandas as pd
from decimal import Decimal
import numpy as np
from datetime import datetime, timedelta
from dotenv import load_dotenv
import yfinance as yf


import matplotlib.pyplot as plt
import plotly.express as px
import pytz  # Make sure to import pytz for timezone handling
import seaborn as sns
from sklearn.linear_model import LinearRegression

import requests
import csv
import json

import warnings

In [34]:
load_dotenv()

API_KEY = os.getenv("alpha_vantage_api_key")

In [35]:
# Addtional setting session
# Set display options to show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")
pd.options.mode.copy_on_write = True

# PE TTM valuation

In [51]:
# Parameters section

alpha_vantage_api_key = API_KEY # FREE TIER API rate limit is 25 requests per day
alpha_vantage_function = {
    'core':[
        'TIME_SERIES_INTRADA'
        ,'TIME_SERIES_DAILY' # this is daily time series quote
        ,'TIME_SERIES_DAILY_ADJUSTED' # this is daily time series adjusted by split/dividend-adjusted
        ,'GLOBAL_QUOTE'
    ]
    ,'fundmental':[
    'INCOME_STATEMENT'
    ,'BALANCE_SHEET' # this is daily time series quote
    ,'CASH_FLOW' # this is daily time series adjusted by split/dividend-adjusted
    ,'EARNINGS'
    ,'EARNINGS_CALENDAR'
]
}

# # Custmized
# ticker_symbols = [

#     'CL'
#     # ,'ACN'
#     # ,'KMB'
#     # ,'GOOG'
#     # ,'TSM'
#     # ,'META'
#     ,'LPG'
#     # ,'NVDA'
#     # ,'SKWD'
#     # ,'MSFT'
#     # ,'NFLX'
#     # ,'V'
#     # ,'PG'
#     # ,'INTU'
#     # ,'ISRG'
#     ]

ticker_symbols = [
 'MO',
 'ADM',
#  'BF.B',
 'BG',
 'CPB',
 'CHD',
 'CLX',
 'KO',
 'CL',
 'CAG',
 'STZ',
 'COST',
 'DG',
 'DLTR',
 'EL',
 'GIS',
 'HSY',
 'HRL',
#  'KVUE',
 'KDP',
 'KMB',
 'KHC',
 'KR',
 'LW',
 'MKC',
 'TAP',
 'MDLZ',
 'MNST',
 'PEP',
 'PM',
 'PG',
 'SJM',
 'SYY',
 'TGT',
 'TSN',
#  'WBA',
 'WMT'
 ]


# Time intelligent parameters
window_days = 90
end_date = datetime.now()
start_date = end_date - timedelta(days=window_days)
earning_calendar = [
    3  # this will return next 1 qtr forecast earning; nowadays the earning calendar only shows the next 1 qtr forecast earning
    ,6  # this will return next 2 qtr forecast earning
    ,12  # this will return next 4 qtr forecast earning
]

PE_yr_range = 9 # this will return x-1 yr PE range, cuz you cannot get current year end PE

ticker_dict_json = {}
ticker_dict_pd = {}

In [52]:
def convert_to_numeric_or_zero(value):
    try:
        # Try to convert the string to a float (or int)
        num = round(float(value), 2)
        return num  # Return the original value if it can be converted to a number
    except ValueError:
        # If conversion fails, return '0'
        return 0

In [53]:
# Daily quote section
for j, symbol in enumerate(ticker_symbols):

    print(j, symbol)
    # STOCK SPLIT FACTOR section
    url = f'https://www.alphavantage.co/query?function=SPLITS&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'data':
            if len(value) > 0:
                stock_split_record_df = pd.DataFrame(value)
                stock_split_record_df['split_factor'] = pd.to_numeric(stock_split_record_df['split_factor'], errors='coerce') # change split_factor series to numeric data
                stock_split_record_df['effective_date'] = pd.to_datetime(stock_split_record_df['effective_date'])
            else:
                # Use pandas index assign instead of direct assign value
                stock_split_record_df = pd.DataFrame({
                'split_factor': [1],
                'effective_date': [datetime.today()]
            })
                # stock_split_record_df = pd.DataFrame()
                # stock_split_record_df['split_factor'] = 1
                # stock_split_record_df['effective_date'] = datetime.today()


    # Daily quote section
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&apikey={alpha_vantage_api_key}&outputsize=full'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'Time Series (Daily)':


            selected_cols = [
                '4. close'
            ]

            Daily_stock_df = pd.DataFrame(value).transpose()[selected_cols] # tranpose the dataframe and sub select selected cols

            # Rename columns
            Daily_stock_df.rename(
                columns={
                    '4. close': 'stock_price'
                    }
                ,inplace=True
                )
            
            Daily_stock_df["stock_price"] = Daily_stock_df["stock_price"].astype(str).apply(lambda x: float(x))
            Daily_stock_df["stock_price"] = Daily_stock_df["stock_price"].round(2)
            Daily_stock_df.index = pd.to_datetime(Daily_stock_df.index)


    for date_i in Daily_stock_df.index.date:
        for date_j in stock_split_record_df['effective_date'].dt.date:
            if date_i == date_j:

                # stock price to divided the split factor
                Daily_stock_df.loc[Daily_stock_df.index.date < date_j, 'stock_price'] /= (stock_split_record_df['split_factor'][stock_split_record_df['effective_date'].dt.date == date_j].values[0])


    # MA200 calculation
    Daily_stock_df['MA200'] = Daily_stock_df.sort_index(ascending=True)['stock_price'].rolling(window=200).mean()

    # Calculate the slope of the MA200 using linear regression
    def calculate_slope(series):
        # Create a time index (0, 1, 2, ..., n) for the linear regression
        x = np.arange(len(series)).reshape(-1, 1)
        y = series.values.reshape(-1, 1)

        # Fit the linear regression model
        model = LinearRegression()
        model.fit(x, y)

        # Return the slope
        return model.coef_[0][0]


    # Apply the slope calculation to the MA200 values
    Daily_stock_df['MA200_slope'] = Daily_stock_df.sort_index(ascending=True)['MA200'].rolling(window=20).apply(calculate_slope, raw=False)



    # Monthly quote section
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'Monthly Time Series':
            Monthly_stock_df = pd.DataFrame(value)

    Monthly_stock_df = Monthly_stock_df.transpose()
    Monthly_stock_df.index = pd.to_datetime(Monthly_stock_df.index)


    filter_1 = (Monthly_stock_df.index.year.isin(range((datetime.today().year - PE_yr_range) ,datetime.today().year)))
    filter_2 = (Monthly_stock_df.index.month == 12) # month = 12 to get the year end closing price

    selected_cols = [
        '4. close'
    ]

    Monthly_stock_df = Monthly_stock_df[
        filter_1
        & filter_2
    ][selected_cols]

    # Rename columns
    Monthly_stock_df.rename(
        columns={
            '4. close': 'stock_price'
            }
        ,inplace=True
        )

    Monthly_stock_df["stock_price"] = Monthly_stock_df["stock_price"].astype(str).apply(lambda x: float(x))
    Monthly_stock_df["stock_price"] = Monthly_stock_df["stock_price"].round(2)

    # modify stock price based on stock split
    for year_i in Monthly_stock_df.index.year:
        for year_j in stock_split_record_df['effective_date'].dt.year:
            if year_i == year_j:

                # stock price to divided the split factor
                Monthly_stock_df.loc[Monthly_stock_df.index.year < year_j, 'stock_price'] /= (stock_split_record_df['split_factor'][stock_split_record_df['effective_date'].dt.year == year_j].values[0])



    # Earning section
    # past earnings from alpha vintage API
    url = f'https://www.alphavantage.co/query?function=EARNINGS&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'annualEarnings':

            selected_cols = [
                'fiscalDateEnding'
                ,'reportedEPS'
            ]

            annualEPS_df = pd.DataFrame(value) # tranpose the dataframe and sub select selected cols


            annualEPS_df['fiscalDateEnding'] = pd.to_datetime(annualEPS_df['fiscalDateEnding']).dt.year

            annualEPS_df = annualEPS_df[
                annualEPS_df['fiscalDateEnding'].isin(
                    range(
                        (datetime.today().year - PE_yr_range) 
                        ,datetime.today().year
                            )
                            )
                            ]

            # Convert the column to decimal type
            for col in selected_cols:
                if col in ['reportedEPS']:
                    annualEPS_df[f'{col}'] = annualEPS_df[f'{col}'].astype(str).apply(lambda x: float(x))

                else:
                    continue
            
            # clean annualEPS_df
            annualEPS_df = annualEPS_df.sort_values('reportedEPS', ascending=False).drop_duplicates('fiscalDateEnding')
            annualEPS_df = annualEPS_df.sort_values('fiscalDateEnding', ascending=False).reset_index(drop=True)

            # calculate metrics
            if Monthly_stock_df.shape[0] <= annualEPS_df.shape[0]: # if Monthly_stock_df has less records than annualEPS_df, choose the mini length record
                annualEPS_df = annualEPS_df[:(Monthly_stock_df.shape[0])]

                annualEPS_df["PE"] = Monthly_stock_df["stock_price"].values / annualEPS_df['reportedEPS'].values
                annualEPS_df[f"PE_{PE_yr_range-1}yr_avg"] = annualEPS_df["PE"].mean().round(2)
                annualEPS_df[f"PE_{PE_yr_range-1}yr_std"] = np.std(annualEPS_df["PE"]).round(2)
                annualEPS_df[f"PE_{PE_yr_range-1}yr_volatility_+"] = (annualEPS_df[f"PE_{PE_yr_range-1}yr_avg"] + annualEPS_df[f"PE_{PE_yr_range-1}yr_std"]).round(2) # 这个是PE的波动范围上限
                annualEPS_df[f"PE_{PE_yr_range-1}yr_volatility_-"] = (annualEPS_df[f"PE_{PE_yr_range-1}yr_avg"] - annualEPS_df[f"PE_{PE_yr_range-1}yr_std"]).round(2) # 这个是PE的波动范围下限



        if key == 'quarterlyEarnings':

            selected_cols = [
                'reportedDate'
                ,'reportedEPS'
            ]

            qtrEPS_df = pd.DataFrame(value)[selected_cols] # tranpose the dataframe and sub select selected cols
            qtrEPS_df['reportedDate'] = pd.to_datetime(qtrEPS_df['reportedDate'])

            # Convert the column to decimal type
            for col in selected_cols:
                if col in ['reportedEPS']:
                    qtrEPS_df[col] = qtrEPS_df[col].astype(str).apply(lambda x: float(x) if x not in [None, 'None', 'nan', 'NaN'] else float(0))
                else:
                    continue


    # # forecast 1 qtr earnings from alpha vantage API -- for now we use yf API
    # # for i in earning_calendar: comment out the for loop in case of future usage, i can be the parameter of {}month
    # CSV_URL = f'https://www.alphavantage.co/query?function=EARNINGS_CALENDAR&symbol={symbol}&horizon=12month&apikey={alpha_vantage_api_key}'
    # with requests.Session() as s:
    #     download = s.get(CSV_URL)
    #     decoded_content = download.content.decode('utf-8')
    #     cr = csv.reader(decoded_content.splitlines(), delimiter=',')
    #     my_list = list(cr)

    #     forecast_earanings_df = pd.DataFrame(
    #         columns=my_list[0]
    #         ,data=my_list[1::]
    #         )
        
    #     if forecast_earanings_df['estimate'].head(1).values != '':
    #         latest_projected_EPS = float(forecast_earanings_df['estimate'].head(1).values)
    #     else:
    #         latest_projected_EPS = 0


    # forecast 1 qtr earning, 1yr earning from yf API
    yf_symbol = yf.Ticker(symbol)
    stock_eps_forecast_df = yf_symbol.earnings_estimate
    stock_eps_forecast_df['period'] = stock_eps_forecast_df.index
    next_qtr_EPS = stock_eps_forecast_df[stock_eps_forecast_df['period'] == '+1q']['avg'].values[0]
    # next yr eps
    next_yr_EPS = stock_eps_forecast_df[stock_eps_forecast_df['period'] == '+1y']['avg'].values[0]


    # EPS trend from yf API
    stock_eps_trend_df = yf_symbol.eps_trend
    stock_eps_trend_df['period'] = stock_eps_trend_df.index
    next_yr_days7ago_EPS = stock_eps_trend_df[stock_eps_trend_df['period'] == '+1y']['7daysAgo'].values[0]
    next_yr_days30ago_EPS = stock_eps_trend_df[stock_eps_trend_df['period'] == '+1y']['30daysAgo'].values[0]
    next_yr_days60ago_EPS = stock_eps_trend_df[stock_eps_trend_df['period'] == '+1y']['60daysAgo'].values[0]
    next_yr_days90ago_EPS = stock_eps_trend_df[stock_eps_trend_df['period'] == '+1y']['90daysAgo'].values[0]



    # stock growth rate from yf API
    stock_growth_est_df = yf_symbol.growth_estimates
    stock_growth_est_df['period'] = stock_growth_est_df.index
    curr_yr_growthrate_symbol = stock_growth_est_df[stock_growth_est_df['period'] == '0y']['stockTrend'].values[0] * 100
    next_yr_growthrate_symbol = stock_growth_est_df[stock_growth_est_df['period'] == '+1y']['stockTrend'].values[0] * 100
    curr_yr_growthrate_index = stock_growth_est_df[stock_growth_est_df['period'] == '0y']['indexTrend'].values[0] * 100
    next_yr_growthrate_index = stock_growth_est_df[stock_growth_est_df['period'] == '+1y']['indexTrend'].values[0] * 100


    # US Treasury section
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=TREASURY_YIELD&interval=daily&maturity=10year&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'data':
            US_T_10yrs_df = pd.DataFrame(value)
            US_T_10yrs_df['value'] = pd.to_numeric(US_T_10yrs_df['value'], errors='coerce') # change dataframe value to numeric data
            US_T_10yrs_YTM = US_T_10yrs_df['value'][0]



    # COMPANY OVERVIEW
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    
    for key, value in data.items():
        if key  == 'MarketCapitalization':
            stock_mkt_cap = float(value)



    # INCOME STATEMENT
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()


    for key, value in data.items():
        if key == 'annualReports':
            annual_income_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            annual_income_df = annual_income_df.sort_values('fiscalDateEnding', ascending=True)

            # annual measurements cols
            annual_income_metric_cols = annual_income_df.columns[1:]

            # Calculate quarter-over-quarter change percentage
            for column in annual_income_metric_cols:  # Exclude the 'fiscalDateEnding' column
                annual_income_df[column] = pd.to_numeric(annual_income_df[column], errors='coerce')
                annual_income_df[f'{column}_YoY'] = annual_income_df[column].pct_change() * 100 
            
            annual_income_YoY_metric_cols = [col for col in annual_income_df.columns if 'YoY' in col]

            # ratios of income statement calculation
            annual_income_df['gross_margin_%'] = (annual_income_df['grossProfit'] / annual_income_df['totalRevenue']) * 100
            annual_income_df['operating_margin_%'] = (annual_income_df['operatingIncome'] / annual_income_df['totalRevenue']) * 100
            annual_income_df['net_margin_%'] = (annual_income_df['netIncome'] / annual_income_df['totalRevenue']) * 100



        if key == 'quarterlyReports':
            qtr_income_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            qtr_income_df = qtr_income_df.sort_values('fiscalDateEnding', ascending=True)

            # qtr measurements cols
            qtr_income_metric_cols = qtr_income_df.columns[1:]
            
            # Calculate quarter-over-quarter change percentage
            for column in qtr_income_df.columns[1:]:  # Exclude the 'fiscalDateEnding' column
                qtr_income_df[column] = pd.to_numeric(qtr_income_df[column], errors='coerce')
                qtr_income_df[f'{column}_QoQ'] = qtr_income_df[column].pct_change() * 100

            qtr_income_QoQ_metric_cols = [col for col in qtr_income_df.columns if 'QoQ' in col]

            # ratios of income statement calculation
            qtr_income_df['gross_margin_%'] = (qtr_income_df['grossProfit'] / qtr_income_df['totalRevenue']) * 100
            qtr_income_df['operating_margin_%'] = (qtr_income_df['operatingIncome'] / qtr_income_df['totalRevenue']) * 100
            qtr_income_df['net_margin_%'] = (qtr_income_df['netIncome'] / qtr_income_df['totalRevenue']) * 100

    income_ratio_cols = [
        'gross_margin_%'
        ,'operating_margin_%'
        ,'net_margin_%'
        ,'netIncome'
        ]
    
    

    # BALANCESHEET
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=BALANCE_SHEET&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():

        if key == 'annualReports':
            annual_balancesheet_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            annual_balancesheet_df = annual_balancesheet_df.sort_values('fiscalDateEnding', ascending=True)

            for column in annual_balancesheet_df.columns[1:]:  # Exclude the 'fiscalDateEnding' column
                annual_balancesheet_df[column] = pd.to_numeric(annual_balancesheet_df[column], errors='coerce')

            annual_balancesheet_df['current_ratio'] = (annual_balancesheet_df['totalCurrentAssets'] / annual_balancesheet_df['totalCurrentLiabilities'])
            annual_balancesheet_df['working_capital'] = annual_balancesheet_df['totalCurrentAssets'] - annual_balancesheet_df['totalCurrentLiabilities']
            annual_balancesheet_df['longTermDebt_to_workingCp_ratio'] = (annual_balancesheet_df['longTermDebt'] / annual_balancesheet_df['working_capital'])
            annual_balancesheet_df['debtEquity_ratio'] = (annual_balancesheet_df['totalLiabilities'] / annual_balancesheet_df['totalShareholderEquity'])
            annual_balancesheet_df['quick_ratio'] = ((annual_balancesheet_df['totalCurrentAssets'] - annual_balancesheet_df['inventory']) / annual_balancesheet_df['totalCurrentLiabilities'])

            
        if key == 'quarterlyReports':
            qtr_balancesheet_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            qtr_balancesheet_df = qtr_balancesheet_df.sort_values('fiscalDateEnding', ascending=True)

            for column in qtr_balancesheet_df.columns[1:]:  # Exclude the 'fiscalDateEnding' column
                qtr_balancesheet_df[column] = pd.to_numeric(qtr_balancesheet_df[column], errors='coerce')

    
            qtr_balancesheet_df['current_ratio'] = (qtr_balancesheet_df['totalCurrentAssets'] / qtr_balancesheet_df['totalCurrentLiabilities'])
            qtr_balancesheet_df['working_capital'] = qtr_balancesheet_df['totalCurrentAssets'] - qtr_balancesheet_df['totalCurrentLiabilities']
            qtr_balancesheet_df['longTermDebt_to_workingCp_ratio'] = (qtr_balancesheet_df['longTermDebt'] / qtr_balancesheet_df['working_capital'])
            qtr_balancesheet_df['debtEquity_ratio'] = (qtr_balancesheet_df['totalLiabilities'] / qtr_balancesheet_df['totalShareholderEquity'])
            qtr_balancesheet_df['quick_ratio'] = ((qtr_balancesheet_df['totalCurrentAssets'] - qtr_balancesheet_df['inventory']) / qtr_balancesheet_df['totalCurrentLiabilities'])

            qtr_balancesheet_df['BVPS_latest'] = round(
                qtr_balancesheet_df.tail(1)['totalShareholderEquity'].sum() / qtr_balancesheet_df.tail(1)['commonStockSharesOutstanding']
                ,2
                )



    balancesheet_ratio_cols = [
        'current_ratio'
        ,'working_capital'
        ,'longTermDebt_to_workingCp_ratio'
        ,'debtEquity_ratio'
        ,'quick_ratio'
        ,'totalShareholderEquity'
    ]


    # CASHFLOW STATEMENT
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=CASH_FLOW&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():

        if key == 'annualReports':
            annual_cashflow_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            annual_cashflow_df = annual_cashflow_df.sort_values('fiscalDateEnding', ascending=True)

            for column in annual_cashflow_df.columns[1:]:  # Exclude the 'fiscalDateEnding' column
                annual_cashflow_df[column] = pd.to_numeric(annual_cashflow_df[column], errors='coerce')
                annual_cashflow_df[f'{column}_YoY'] = annual_cashflow_df[column].pct_change() * 100

            
        if key == 'quarterlyReports':
            qtr_cashflow_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            qtr_cashflow_df = qtr_cashflow_df.sort_values('fiscalDateEnding', ascending=True)

            for column in qtr_cashflow_df.columns[1:]:  # Exclude the 'fiscalDateEnding' column
                qtr_cashflow_df[column] = pd.to_numeric(qtr_cashflow_df[column], errors='coerce')
                qtr_cashflow_df[f'{column}_YoY'] = qtr_cashflow_df[column].pct_change() * 100


    stock_ratios_annual_consolidate_df = pd.DataFrame()
    stock_ratios_qtr_consolidate_df = pd.DataFrame()

    stock_ratios_annual_consolidate_df[income_ratio_cols] = annual_income_df[income_ratio_cols]
    stock_ratios_annual_consolidate_df[balancesheet_ratio_cols] = annual_balancesheet_df[balancesheet_ratio_cols]

    stock_ratios_qtr_consolidate_df[income_ratio_cols] = qtr_income_df[income_ratio_cols]
    stock_ratios_qtr_consolidate_df[balancesheet_ratio_cols] = qtr_balancesheet_df[balancesheet_ratio_cols]

    # calculating new consolidated metrics 
    # annual df
    stock_ratios_annual_consolidate_df['ROE_%'] = (annual_income_df['netIncome'] / annual_balancesheet_df['totalShareholderEquity']) * 100
    stock_ratios_annual_consolidate_df['liquidation_value'] = ((annual_balancesheet_df['totalAssets'] - annual_balancesheet_df['intangibleAssets']) - annual_balancesheet_df['totalLiabilities'])
    # stock_ratios_annual_consolidate_df['liquidation_mktcap_ratio_%'] = round((stock_ratios_annual_consolidate_df['liquidation_value'] / stock_mkt_cap) * 100, 2)
    stock_ratios_annual_consolidate_df['fiscalDateEnding'] = annual_balancesheet_df['fiscalDateEnding']

    # qtr df
    stock_ratios_qtr_consolidate_df['ROE_%'] = (qtr_income_df['netIncome'] / qtr_balancesheet_df['totalShareholderEquity']) * 100
    stock_ratios_qtr_consolidate_df['liquidation_value'] = ((qtr_balancesheet_df['totalAssets'] - qtr_balancesheet_df['intangibleAssets']) - qtr_balancesheet_df['totalLiabilities'])
    # stock_ratios_qtr_consolidate_df['liquidation_mktcap_ratio_%'] = round((stock_ratios_qtr_consolidate_df['liquidation_value'] / stock_mkt_cap) * 100, 2)
    stock_ratios_qtr_consolidate_df['fiscalDateEnding'] = qtr_balancesheet_df['fiscalDateEnding']





    # Consolidated section
    stock_consolidate_df = Daily_stock_df.head(window_days)


    stock_consolidate_df_date = stock_consolidate_df.index
    for i in stock_consolidate_df_date:
                
        # Filter the DataFrame to include only dates(index) less than or equal to the target date
        filtered_qtrEPS_df = qtrEPS_df[qtrEPS_df['reportedDate'] < i]

        # Select the first four rows from the past_qtrs_EPS
        past_4_qtrs_EPS = filtered_qtrEPS_df.head(4) 
        past_3_qtrs_EPS = filtered_qtrEPS_df.head(3)
        past_1_qtr_EPS = filtered_qtrEPS_df.head(1)

        # Calculate the sum of the numeric values in the selected rows
        EPS_TTM = past_4_qtrs_EPS['reportedEPS'].values.sum()
        EPS_curr_qtr = past_1_qtr_EPS['reportedEPS'].values.sum()

        # assign each index row with the EPS_TTM
        stock_consolidate_df.loc[i, "EPS_TTM"] = EPS_TTM
        stock_consolidate_df.loc[i, "EPS_currentQtr"] = EPS_curr_qtr

        if i == max(stock_consolidate_df.index):
            EPS_nextQtr_projected = next_qtr_EPS + past_3_qtrs_EPS['reportedEPS'].values.sum()  # This metrics is the past 3 qtrs post EPS + 1 projected EPS
        else:
            continue

        stock_consolidate_df["EPS_nextQtr"] = next_qtr_EPS
        stock_consolidate_df["EPS_nextQtr_TTM"] = EPS_nextQtr_projected
        stock_consolidate_df["EPS_nextYr"] = next_yr_EPS

    

    # stock's stats
    stock_consolidate_df["Ticker"] = symbol
    # stock_consolidate_df["PE_1yr_forward"] = round(PE_12month_projected, 2)
    stock_consolidate_df["PE_TTM"] = (stock_consolidate_df["stock_price"] / stock_consolidate_df["EPS_TTM"]).round(2)
    stock_consolidate_df["PE_TTM_avg"] = stock_consolidate_df["PE_TTM"].mean().round(2)
    stock_consolidate_df["PE_TTM_std"] = np.std(stock_consolidate_df["PE_TTM"]).round(2)
    stock_consolidate_df["PE_TTM_volatility_+"] = (stock_consolidate_df["PE_TTM_avg"] + stock_consolidate_df["PE_TTM_std"]).round(2) # 这个是PE的波动范围上限
    stock_consolidate_df["PE_TTM_volatility_-"] = (stock_consolidate_df["PE_TTM_avg"] - stock_consolidate_df["PE_TTM_std"]).round(2) # 这个是PE的波动范围下限

    stock_consolidate_df["relative_valuation_TTM_+"] = (stock_consolidate_df["PE_TTM_volatility_+"] * stock_consolidate_df["EPS_TTM"]).round(2) # 这个是relative valuation的价格上限
    stock_consolidate_df["relative_valuation_TTM_-"] = (stock_consolidate_df["PE_TTM_volatility_-"] * stock_consolidate_df["EPS_TTM"]).round(2) # 这个是relative valuation的价格下限
    stock_consolidate_df["relative_valuation_TTM_median"] = (np.median([stock_consolidate_df["relative_valuation_TTM_+"][0], stock_consolidate_df["relative_valuation_TTM_-"][0]])).round(2) #这个是根据最新TTM PE估值的价格中位数

    stock_consolidate_df["relative_valuation_nextQuater_projected_+"] = (stock_consolidate_df["PE_TTM_volatility_+"] * stock_consolidate_df["EPS_nextQtr_TTM"]).round(2) # 这个是relative valuation的价格上限
    stock_consolidate_df["relative_valuation_nextQuater_projected_-"] = (stock_consolidate_df["PE_TTM_volatility_-"] * stock_consolidate_df["EPS_nextQtr_TTM"]).round(2) # 这个是relative valuation的价格下限
    stock_consolidate_df["relative_valuation_nextQuater_projected_median"] = (np.median([stock_consolidate_df["relative_valuation_nextQuater_projected_+"][0], stock_consolidate_df["relative_valuation_nextQuater_projected_-"][0]])).round(2) #这个是根据3 qtrs post EPS + 1 projected EPS 得出PE估值的价格中位数

    stock_consolidate_df[f"{window_days}_price_min"] = stock_consolidate_df["stock_price"].min().round(2)
    stock_consolidate_df[f"{window_days}_price_max"] = stock_consolidate_df["stock_price"].max().round(2)
    stock_consolidate_df[f"{window_days}_price_avg"] = stock_consolidate_df["stock_price"].mean().round(2)
    stock_consolidate_df[f"{window_days}_price_std"] = np.std(stock_consolidate_df["stock_price"]).round(2)

    stock_consolidate_df[f"PE_{PE_yr_range-1}yr_avg"] = annualEPS_df[f"PE_{PE_yr_range-1}yr_avg"].values[0]
    stock_consolidate_df[f"PE_{PE_yr_range-1}yr_std"] = annualEPS_df[f"PE_{PE_yr_range-1}yr_std"].values[0]
    stock_consolidate_df[f"PE_{PE_yr_range-1}yr_volatility_+"] = annualEPS_df[f"PE_{PE_yr_range-1}yr_volatility_+"].values[0]
    stock_consolidate_df[f"PE_{PE_yr_range-1}yr_volatility_-"] = annualEPS_df[f"PE_{PE_yr_range-1}yr_volatility_-"].values[0]


    stock_consolidate_df["relative_valuation_nextYear_projected_+"] = (stock_consolidate_df["PE_TTM_volatility_+"] * next_yr_EPS).round(2) # 这个是relative valuation的价格上限
    stock_consolidate_df["relative_valuation_nextYear_projected_-"] = (stock_consolidate_df["PE_TTM_volatility_-"] * next_yr_EPS).round(2) # 这个是relative valuation的价格下限
    stock_consolidate_df["relative_valuation_nextYear_projected_median"] = (np.median([stock_consolidate_df["relative_valuation_nextYear_projected_+"][0], stock_consolidate_df["relative_valuation_nextYear_projected_-"][0]])).round(2) #这个是根据next year projected EPS 得出PE估值的价格中位数

    # stock_consolidate_df["PEG_next12months"] = PEG_12month_projected
    stock_consolidate_df["EPS_nextYr"] = round(next_yr_EPS, 2)
    # stock_consolidate_df["PEG_TTM"] = (stock_consolidate_df["PE_TTM"] / (((next_yr_EPS - stock_consolidate_df["EPS_TTM"]) / stock_consolidate_df["EPS_TTM"]) * 100)).round(2) # 这个是截止下一年的EPS growth rate所得出的PEG ratio, <1是undervalue的表现
    
    stock_consolidate_df["EPS_nextYr_growthRate"] = (((next_yr_EPS - stock_consolidate_df["EPS_TTM"]) / stock_consolidate_df["EPS_TTM"]) * 100).round(2)
    stock_consolidate_df["EPS_nextQtr_growthRate"] = (((stock_consolidate_df["EPS_nextQtr_TTM"] - stock_consolidate_df["EPS_TTM"]) / stock_consolidate_df["EPS_TTM"]) * 100).round(2)

    stock_consolidate_df["EarningYield_TTM"] = ((stock_consolidate_df["EPS_TTM"] / stock_consolidate_df["stock_price"]) * 100).round(2)
    stock_consolidate_df["ERP_TTM"] = stock_consolidate_df["EarningYield_TTM"] - US_T_10yrs_YTM # it means a comparison between equity return and 10 years risk free, usually ERP >= 3 for short term invest at least, ERP >= 5 for long term invest 

    # stock_consolidate_df['latest_qtr_liquidation_mktcap_ratio_%'] = stock_ratios_qtr_consolidate_df['liquidation_mktcap_ratio_%'].values[-1]
    stock_consolidate_df['BVPS_latest'] = qtr_balancesheet_df['BVPS_latest'].values[-1]

    stock_consolidate_df['MA200_slope'] = Daily_stock_df['MA200_slope']

    stock_consolidate_df['FCF_per_share_TTM'] = round((qtr_cashflow_df['operatingCashflow'] - qtr_cashflow_df['capitalExpenditures']).tail(4).sum() / qtr_balancesheet_df['commonStockSharesOutstanding'].values[-1], 2)
    stock_consolidate_df['PFCF_TTM'] = round(stock_consolidate_df['stock_price'][0]  / stock_consolidate_df['FCF_per_share_TTM'], 2)
    stock_consolidate_df['FCF_yield_TTM'] = round((stock_consolidate_df['FCF_per_share_TTM'] / stock_consolidate_df['stock_price'][0]) * 100, 2) # % return on cash of every single dollar you spent on stock price, the higher the better

    # EPS trend for next year estimates
    stock_consolidate_df['next_yr_days7ago_EPS'] = round(next_yr_days7ago_EPS, 2)
    stock_consolidate_df['next_yr_days30ago_EPS'] = round(next_yr_days30ago_EPS, 2)
    stock_consolidate_df['next_yr_days60ago_EPS'] = round(next_yr_days60ago_EPS, 2)
    stock_consolidate_df['next_yr_days90ago_EPS'] = round(next_yr_days90ago_EPS, 2)

    # symbol growth rate
    stock_consolidate_df['curr_yr_growthrate_symbol'] = round(curr_yr_growthrate_symbol, 2)
    stock_consolidate_df['next_yr_growthrate_symbol'] = round(next_yr_growthrate_symbol, 2)
    stock_consolidate_df['curr_yr_growthrate_index'] = round(curr_yr_growthrate_index, 2)
    stock_consolidate_df['next_yr_growthrate_index'] = round(next_yr_growthrate_index, 2)

    # filter conditions
    conditions = [
    (stock_consolidate_df["stock_price"] < stock_consolidate_df["relative_valuation_TTM_-"]),
    (stock_consolidate_df["stock_price"] > stock_consolidate_df["relative_valuation_TTM_+"]),
    ((stock_consolidate_df["stock_price"] >= stock_consolidate_df["relative_valuation_TTM_-"]) & (stock_consolidate_df["stock_price"] <= stock_consolidate_df["relative_valuation_TTM_+"])),
    ]

    categories = [
        'undervalued'
        ,'overvalued'
        ,'fair'
        ]

    # This KPI assess if the current stock price is under/over/fair to the current relative valuation
    stock_consolidate_df["curr_assessment"] = None

    for condition, category in zip(conditions, categories):
        stock_consolidate_df.loc[condition, "price_valuation_assessment"] = category





    # Append key-value pairs to the dictionary
    selected_cols = [
    "Ticker"
    ,"stock_price"
    ,"EPS_currentQtr"
    ,"EPS_nextQtr"
    ,"EPS_TTM"
    ,"EPS_nextQtr_TTM"
    ,"EPS_nextYr"
    # ,"PE_1yr_forward"
    ,"PE_TTM"
    ,"PE_TTM_avg"
    ,"PE_TTM_volatility_+"
    ,"PE_TTM_volatility_-"
    ,f"PE_{PE_yr_range-1}yr_avg"
    ,f"PE_{PE_yr_range-1}yr_volatility_+"
    ,f"PE_{PE_yr_range-1}yr_volatility_-"
    ,"relative_valuation_TTM_+"
    ,"relative_valuation_TTM_-"
    ,"relative_valuation_TTM_median"
    ,"relative_valuation_nextQuater_projected_+"
    ,"relative_valuation_nextQuater_projected_-"
    ,"relative_valuation_nextQuater_projected_median"
    ,"relative_valuation_nextYear_projected_+"
    ,"relative_valuation_nextYear_projected_-"
    ,"relative_valuation_nextYear_projected_median"
    ,"price_valuation_assessment"
    ,"EPS_nextQtr_growthRate"
    ,"EPS_nextYr_growthRate"
    # ,"PEG_next12months"
    # ,"PEG_TTM"
    ,"EarningYield_TTM"
    ,"ERP_TTM"
    # ,"latest_qtr_liquidation_mktcap_ratio_%"
    ,"BVPS_latest"
    ,"PFCF_TTM"
    ,"FCF_yield_TTM" 
    # ,"MA200_slope"
    ,'next_yr_days7ago_EPS'
    ,'next_yr_days30ago_EPS'
    ,'next_yr_days60ago_EPS'
    ,'next_yr_days90ago_EPS'
    ,'curr_yr_growthrate_symbol'
    ,'next_yr_growthrate_symbol'
    ,'curr_yr_growthrate_index'
    ,'next_yr_growthrate_index'
    ]


    # store each stock info as pd into dictionary
    ticker_dict_pd[symbol] = stock_consolidate_df[selected_cols]
    # transfer pandas dataframe to json format, and each stock info into dictionary
    ticker_dict_json[symbol] = stock_consolidate_df[selected_cols].to_dict()

    if j == 0:
        # screener df creation
        # screener df will store each stock's consolidate df's first row and union them together for screening purposee
        ticker_screen_df = pd.DataFrame(
            columns=selected_cols
        ) 
        stock_consolidate_df_values = stock_consolidate_df[selected_cols].values[0]

        # Insert rows into the DataFrame
        ticker_screen_df.loc[j] = stock_consolidate_df_values

    else:
        stock_consolidate_df_values = stock_consolidate_df[selected_cols].values[0]

        
        ticker_screen_df.loc[j] = stock_consolidate_df_values

# ticker screen df consolidated metrics
ticker_screen_df['Industry_PE_TTM_avg'] = round(ticker_screen_df['PE_TTM'].mean(), 2)

0 MO
1 ADM
2 BG
3 CPB
4 CHD
5 CLX
6 KO
7 CL
8 CAG
9 STZ
10 COST
11 DG
12 DLTR
13 EL
14 GIS
15 HSY
16 HRL
17 KDP
18 KMB
19 KHC
20 KR
21 LW
22 MKC
23 TAP
24 MDLZ
25 MNST
26 PEP
27 PM
28 PG
29 SJM
30 SYY
31 TGT
32 TSN
33 WMT


In [39]:
ticker_dict_pd['LPG']

,Ticker,stock_price,EPS_currentQtr,EPS_nextQtr,EPS_TTM,EPS_nextQtr_TTM,EPS_nextYr,PE_TTM,PE_TTM_avg,PE_TTM_volatility_+,PE_TTM_volatility_-,PE_8yr_avg,PE_8yr_volatility_+,PE_8yr_volatility_-,relative_valuation_TTM_+,relative_valuation_TTM_-,relative_valuation_TTM_median,relative_valuation_nextQuater_projected_+,relative_valuation_nextQuater_projected_-,relative_valuation_nextQuater_projected_median,relative_valuation_nextYear_projected_+,relative_valuation_nextYear_projected_-,relative_valuation_nextYear_projected_median,price_valuation_assessment,EPS_nextQtr_growthRate,EPS_nextYr_growthRate,EarningYield_TTM,ERP_TTM,BVPS_latest,PFCF_TTM,FCF_yield_TTM,next_yr_days7ago_EPS,next_yr_days30ago_EPS,next_yr_days60ago_EPS,next_yr_days90ago_EPS,curr_yr_growthrate_symbol,next_yr_growthrate_symbol,curr_yr_growthrate_index,next_yr_growthrate_index
2026-02-05,LPG,30.06,1.11,0.76,2.94,3.45,2.91,10.22,14.66,19.46,9.86,0.67,11.96,-10.62,57.21,28.99,43.1,67.14,34.02,50.58,56.68,28.72,42.7,fair,17.35,-0.94,9.78,5.49,25.01,14.38,6.95,2.69,2.58,2.55,2.56,84.03,-30.28,16.06,15.46
2026-02-04,LPG,29.72,1.11,0.76,2.94,3.45,2.91,10.11,14.66,19.46,9.86,0.67,11.96,-10.62,57.21,28.99,43.1,67.14,34.02,50.58,56.68,28.72,42.7,fair,17.35,-0.94,9.89,5.60,25.01,14.38,6.95,2.69,2.58,2.55,2.56,84.03,-30.28,16.06,15.46
2026-02-03,LPG,30.36,1.11,0.76,2.94,3.45,2.91,10.33,14.66,19.46,9.86,0.67,11.96,-10.62,57.21,28.99,43.1,67.14,34.02,50.58,56.68,28.72,42.7,fair,17.35,-0.94,9.68,5.39,25.01,14.38,6.95,2.69,2.58,2.55,2.56,84.03,-30.28,16.06,15.46
2026-02-02,LPG,29.83,1.11,0.76,2.94,3.45,2.91,10.15,14.66,19.46,9.86,0.67,11.96,-10.62,57.21,28.99,43.1,67.14,34.02,50.58,56.68,28.72,42.7,fair,17.35,-0.94,9.86,5.57,25.01,14.38,6.95,2.69,2.58,2.55,2.56,84.03,-30.28,16.06,15.46
2026-01-30,LPG,29.53,1.11,0.76,2.94,3.45,2.91,10.04,14.66,19.46,9.86,0.67,11.96,-10.62,57.21,28.99,43.1,67.14,34.02,50.58,56.68,28.72,42.7,fair,17.35,-0.94,9.96,5.67,25.01,14.38,6.95,2.69,2.58,2.55,2.56,84.03,-30.28,16.06,15.46
2026-01-29,LPG,29.74,1.31,0.76,2.26,3.45,2.91,13.16,14.66,19.46,9.86,0.67,11.96,-10.62,43.98,22.28,43.1,67.14,34.02,50.58,56.68,28.72,42.7,fair,52.65,28.87,7.60,3.31,25.01,14.38,6.95,2.69,2.58,2.55,2.56,84.03,-30.28,16.06,15.46
2026-01-28,LPG,28.93,1.31,0.76,2.26,3.45,2.91,12.80,14.66,19.46,9.86,0.67,11.96,-10.62,43.98,22.28,43.1,67.14,34.02,50.58,56.68,28.72,42.7,fair,52.65,28.87,7.81,3.52,25.01,14.38,6.95,2.69,2.58,2.55,2.56,84.03,-30.28,16.06,15.46
2026-01-27,LPG,27.93,1.31,0.76,2.26,3.45,2.91,12.36,14.66,19.46,9.86,0.67,11.96,-10.62,43.98,22.28,43.1,67.14,34.02,50.58,56.68,28.72,42.7,fair,52.65,28.87,8.09,3.80,25.01,14.38,6.95,2.69,2.58,2.55,2.56,84.03,-30.28,16.06,15.46
2026-01-26,LPG,27.28,1.31,0.76,2.26,3.45,2.91,12.07,14.66,19.46,9.86,0.67,11.96,-10.62,43.98,22.28,43.1,67.14,34.02,50.58,56.68,28.72,42.7,fair,52.65,28.87,8.28,3.99,25.01,14.38,6.95,2.69,2.58,2.55,2.56,84.03,-30.28,16.06,15.46
2026-01-23,LPG,27.52,1.31,0.76,2.26,3.45,2.91,12.18,14.66,19.46,9.86,0.67,11.96,-10.62,43.98,22.28,43.1,67.14,34.02,50.58,56.68,28.72,42.7,fair,52.65,28.87,8.21,3.92,25.01,14.38,6.95,2.69,2.58,2.55,2.56,84.03,-30.28,16.06,15.46


# Stock screener

In [54]:
ticker_screen_df = ticker_screen_df.sort_values(by=['price_valuation_assessment']
                                                ,ascending=False)

selected_cols = [
    'Ticker'

    ,'BVPS_latest'
    ,'stock_price'
    # Earnings
    ,'EPS_TTM'
    ,"EPS_nextYr"

    # PE relative valuation
    ,'PE_TTM'
    ,'PE_TTM_avg'
    ,f"PE_{PE_yr_range-1}yr_avg"
    ,f"PE_{PE_yr_range-1}yr_volatility_+"
    ,f"PE_{PE_yr_range-1}yr_volatility_-"
    ,"relative_valuation_TTM_+"
    ,"relative_valuation_TTM_-"
    ,"relative_valuation_TTM_median"
    ,"relative_valuation_nextQuater_projected_+"
    ,"relative_valuation_nextQuater_projected_-"
    ,"relative_valuation_nextQuater_projected_median"
    ,"relative_valuation_nextYear_projected_+"
    ,"relative_valuation_nextYear_projected_-"
    ,"relative_valuation_nextYear_projected_median"
    ,'price_valuation_assessment'

    # Growth
    ,"EPS_nextQtr_growthRate"
    ,"EPS_nextYr_growthRate"
    # ,"PEG_next12months"
    # ,"PEG_TTM"

    # Other valuation metrics
    ,'ERP_TTM'
    # ,'latest_qtr_liquidation_mktcap_ratio_%'
    ,'PFCF_TTM'
    ,'FCF_yield_TTM'
    ,'next_yr_days7ago_EPS'
    ,'next_yr_days30ago_EPS'
    ,'next_yr_days60ago_EPS'
    ,'next_yr_days90ago_EPS'
    ,'curr_yr_growthrate_symbol'
    ,'next_yr_growthrate_symbol'
    ,'curr_yr_growthrate_index'
    ,'next_yr_growthrate_index'
    ,'Industry_PE_TTM_avg'
]

ticker_screen_df[selected_cols]

,Ticker,BVPS_latest,stock_price,EPS_TTM,EPS_nextYr,PE_TTM,PE_TTM_avg,PE_8yr_avg,PE_8yr_volatility_+,PE_8yr_volatility_-,relative_valuation_TTM_+,relative_valuation_TTM_-,relative_valuation_TTM_median,relative_valuation_nextQuater_projected_+,relative_valuation_nextQuater_projected_-,relative_valuation_nextQuater_projected_median,relative_valuation_nextYear_projected_+,relative_valuation_nextYear_projected_-,relative_valuation_nextYear_projected_median,price_valuation_assessment,EPS_nextQtr_growthRate,EPS_nextYr_growthRate,ERP_TTM,PFCF_TTM,FCF_yield_TTM,next_yr_days7ago_EPS,next_yr_days30ago_EPS,next_yr_days60ago_EPS,next_yr_days90ago_EPS,curr_yr_growthrate_symbol,next_yr_growthrate_symbol,curr_yr_growthrate_index,next_yr_growthrate_index,Industry_PE_TTM_avg
13,EL,11.05,99.47,1.95,3.01,51.01,61.92,44.65,59.25,30.05,131.78,109.71,120.74,111.88,93.14,102.51,203.3,169.25,186.28,undervalued,-15.1,54.27,-2.25,31.88,3.14,2.94,2.93,2.92,2.92,45.58,36.85,16.1,15.41,21.88
0,MO,-2.09,65.4,5.42,5.8,12.07,11.23,11.49,15.08,7.9,64.39,57.34,60.86,67.26,59.9,63.58,68.93,61.39,65.16,overvalued,4.45,7.06,4.08,12.09,8.27,5.8,5.79,5.79,5.8,3.12,3.82,16.1,15.41,21.88
15,HSY,22.81,231.53,6.31,9.57,36.69,24.79,23.77,27.35,20.19,170.24,142.61,156.43,156.93,131.46,144.2,258.29,216.36,237.32,overvalued,-7.82,51.72,-1.48,30.38,3.29,8.38,8.19,7.95,8.05,31.65,15.24,16.1,15.41,21.88
32,TSN,52.24,65.26,3.95,4.44,16.52,14.06,15.84,25.22,6.46,58.93,52.14,55.54,60.22,53.27,56.74,66.19,58.56,62.38,overvalued,2.17,12.31,1.84,20.33,4.92,4.41,4.34,4.44,4.65,-4.77,13.07,16.1,15.41,21.88
31,TGT,34.02,115.55,7.54,7.68,15.32,12.73,18.19,24.39,11.99,103.98,87.99,95.98,90.99,77.0,84.0,105.95,89.66,97.8,overvalued,-12.49,1.9,2.32,17.51,5.71,7.67,7.68,7.68,7.92,-17.63,5.28,16.1,15.41,21.88
30,SYY,4.75,86.53,4.58,5.09,18.89,17.03,26.4,37.94,14.86,81.57,74.42,78.0,91.51,83.5,87.5,90.69,82.75,86.72,overvalued,12.19,11.18,1.08,21.58,4.63,5.1,5.05,5.05,5.05,2.96,10.89,16.1,15.41,21.88
29,SJM,56.69,109.51,8.92,7.97,12.28,11.2,13.46,15.89,11.03,103.2,96.6,99.9,104.55,97.86,101.2,92.19,86.29,89.24,overvalued,1.3,-10.67,3.94,18.44,5.42,8.04,8.0,7.78,7.76,-11.19,9.92,16.1,15.41,21.88
28,PG,22.0,159.17,6.79,7.32,23.44,21.5,25.14,27.65,22.63,150.74,141.23,145.99,150.5,141.01,145.76,162.47,152.22,157.34,overvalued,-0.16,7.78,0.06,25.97,3.85,7.31,7.35,7.39,7.39,2.03,5.02,16.1,15.41,21.88
27,PM,-5.34,182.81,7.39,9.11,24.74,21.75,17.31,20.08,14.54,170.04,151.42,160.73,183.44,163.35,173.4,209.58,186.63,198.1,overvalued,7.88,23.25,-0.17,27.16,3.68,9.09,9.11,9.13,9.14,10.73,9.09,16.1,15.41,21.88
26,PEP,14.88,170.49,8.15,9.11,20.92,18.73,23.0,26.57,19.43,157.95,147.35,152.65,173.17,161.55,167.36,176.53,164.69,170.61,overvalued,9.64,11.77,0.57,22.67,4.41,8.99,8.97,8.97,8.98,5.87,5.7,16.1,15.41,21.88


# Price EPS chart

In [40]:
# Parameters section
alpha_vantage_api_key = API_KEY


ticker_symbols = [
    'AXP'
]



In [41]:
# Daily quote section
for j, symbol in enumerate(ticker_symbols):

    print(j, symbol)
    # STOCK SPLIT FACTOR section
    url = f'https://www.alphavantage.co/query?function=SPLITS&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'data':
            if len(value) > 0:
                stock_split_record_df = pd.DataFrame(value)
                stock_split_record_df['split_factor'] = pd.to_numeric(stock_split_record_df['split_factor'], errors='coerce') # change split_factor series to numeric data
                stock_split_record_df['effective_date'] = pd.to_datetime(stock_split_record_df['effective_date'])
            else:
                # Use pandas index assign instead of direct assign value
                stock_split_record_df = pd.DataFrame({
                'split_factor': [1],
                'effective_date': [datetime.today()]
            })


    # Daily quote section
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&apikey={alpha_vantage_api_key}&outputsize=full'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'Time Series (Daily)':


            selected_cols = [
                '4. close'
            ]

            Daily_stock_df = pd.DataFrame(value).transpose()[selected_cols] # tranpose the dataframe and sub select selected cols

            # Rename columns
            Daily_stock_df.rename(
                columns={
                    '4. close': 'stock_price'
                    }
                ,inplace=True
                )
            
            Daily_stock_df["stock_price"] = Daily_stock_df["stock_price"].astype(str).apply(lambda x: float(x))
            Daily_stock_df["stock_price"] = Daily_stock_df["stock_price"].round(2)
            Daily_stock_df.index = pd.to_datetime(Daily_stock_df.index)


    for date_i in Daily_stock_df.index.date:
        for date_j in stock_split_record_df['effective_date'].dt.date:
            if date_i == date_j:

                # stock price to divided the split factor
                Daily_stock_df.loc[Daily_stock_df.index.date < date_j, 'stock_price'] /= (stock_split_record_df['split_factor'][stock_split_record_df['effective_date'].dt.date == date_j].values[0])




    # Earning section
    # past earnings from alpha vintage API
    url = f'https://www.alphavantage.co/query?function=EARNINGS&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'annualEarnings':

            selected_cols = [
                'fiscalDateEnding'
                ,'reportedEPS'
            ]

            annualEPS_df = pd.DataFrame(value) # tranpose the dataframe and sub select selected cols


            annualEPS_df['fiscalDateEnding'] = pd.to_datetime(annualEPS_df['fiscalDateEnding']).dt.year


            # Convert the column to decimal type
            for col in selected_cols:
                if col in ['reportedEPS']:
                    annualEPS_df[f'{col}'] = annualEPS_df[f'{col}'].astype(str).apply(lambda x: float(x))

                else:
                    continue
            
            # clean annualEPS_df
            annualEPS_df = annualEPS_df.sort_values('reportedEPS', ascending=False).drop_duplicates('fiscalDateEnding')
            annualEPS_df = annualEPS_df.sort_values('fiscalDateEnding', ascending=False).reset_index(drop=True)



        if key == 'quarterlyEarnings':

            selected_cols = [
                'reportedDate'
                ,'reportedEPS'
            ]

            qtrEPS_df = pd.DataFrame(value)[selected_cols] # tranpose the dataframe and sub select selected cols
            qtrEPS_df['reportedDate'] = pd.to_datetime(qtrEPS_df['reportedDate'])

            # Convert the column to decimal type
            for col in selected_cols:
                if col in ['reportedEPS']:
                    qtrEPS_df[col] = qtrEPS_df[col].astype(str).apply(lambda x: float(x) if x not in [None, 'None', 'nan', 'NaN'] else float(0))
                else:
                    continue



    # Consolidated section
    stock_consolidate_df = Daily_stock_df
    stock_consolidate_df_date = stock_consolidate_df.index
    for i in stock_consolidate_df_date:
                
        # Filter the DataFrame to include only dates(index) less than or equal to the target date
        filtered_qtrEPS_df = qtrEPS_df[qtrEPS_df['reportedDate'] < i]

        # Select the first four rows from the past_qtrs_EPS
        past_4_qtrs_EPS = filtered_qtrEPS_df.head(4) 
        past_3_qtrs_EPS = filtered_qtrEPS_df.head(3)
        past_1_qtr_EPS = filtered_qtrEPS_df.head(1)

        # Calculate the sum of the numeric values in the selected rows
        EPS_TTM = past_4_qtrs_EPS['reportedEPS'].values.sum()
        EPS_curr_qtr = past_1_qtr_EPS['reportedEPS'].values.sum()

        # assign each index row with the EPS_TTM
        stock_consolidate_df.loc[i, "EPS_TTM"] = EPS_TTM
        stock_consolidate_df.loc[i, "EPS_currentQtr"] = EPS_curr_qtr


    # stock's stats
    stock_consolidate_df["Ticker"] = symbol
    stock_consolidate_df["PE_TTM"] = (stock_consolidate_df["stock_price"] / stock_consolidate_df["EPS_TTM"]).round(2)
    stock_consolidate_df["PE_TTM_avg"] = stock_consolidate_df["PE_TTM"].mean().round(2)
    stock_consolidate_df["PE_TTM_std"] = np.std(stock_consolidate_df["PE_TTM"]).round(2)
    stock_consolidate_df["PE_TTM_volatility_+"] = (stock_consolidate_df["PE_TTM_avg"] + stock_consolidate_df["PE_TTM_std"]).round(2) # 这个是PE的波动范围上限
    stock_consolidate_df["PE_TTM_volatility_-"] = (stock_consolidate_df["PE_TTM_avg"] - stock_consolidate_df["PE_TTM_std"]).round(2) # 这个是PE的波动范围下限

    stock_consolidate_df["relative_valuation_TTM_+"] = (stock_consolidate_df["PE_TTM_volatility_+"] * stock_consolidate_df["EPS_TTM"]).round(2) # 这个是relative valuation的价格上限
    stock_consolidate_df["relative_valuation_TTM_-"] = (stock_consolidate_df["PE_TTM_volatility_-"] * stock_consolidate_df["EPS_TTM"]).round(2) # 这个是relative valuation的价格下限
    stock_consolidate_df["relative_valuation_TTM_median"] = (np.median([stock_consolidate_df["relative_valuation_TTM_+"][0], stock_consolidate_df["relative_valuation_TTM_-"][0]])).round(2) #这个是根据最新TTM PE估值的价格中位数


0 AXP


In [42]:
import plotly.graph_objects as go


# Create the figure
fig = go.Figure()

# Add stock_price on primary y-axis (left)
fig.add_trace(go.Scatter(
    x=stock_consolidate_df.index,
    y=stock_consolidate_df["stock_price"],
    mode='lines',
    line=dict(color='black'),
    name='Stock Price',
    yaxis="y1"
))

# Add EPS_TTM on secondary y-axis (right)
fig.add_trace(go.Scatter(
    x=stock_consolidate_df.index,
    y=stock_consolidate_df["EPS_TTM"],
    mode='lines',
    fill='tonexty',  # Shadow effect
    line=dict(color='green'),
    name='EPS_TTM',
    yaxis="y2"
))

# Add PE_TTM on secondary y-axis (right)
fig.add_trace(go.Scatter(
    x=stock_consolidate_df.index,
    y=stock_consolidate_df["PE_TTM"],
    mode='lines',
    line=dict(color='blue', dash='dot'),
    name='PE_TTM',
    yaxis="y2"
))

# Update layout to remove grid and configure dual y-axes
fig.update_layout(
    title="Stock Metrics Over Time",
    xaxis=dict(title="Date", showgrid=False),
    yaxis=dict(
        title="Stock Price",
        showgrid=False,
        titlefont=dict(color="black"),
        tickfont=dict(color="black")
    ),
    yaxis2=dict(
        title="EPS_TTM & PE_TTM",
        overlaying="y",  # Overlay with y1
        side="right",
        showgrid=False,
        titlefont=dict(color="blue"),
        tickfont=dict(color="blue")
    ),
    legend_title="Metrics",
    template="plotly_white",
    plot_bgcolor='white'  # Background color
)

# Show the plot
fig.show()

# Financial statement of single stock

In [63]:
stock_financial_dict = {} # the dict will store ticker as key, the string value of 'annual_df' and 'qtr_df' as nested key, actual json as value

ticker_symbols = [
    'NFLX'
    ]
# ticker_symbols = electric_utility_symbols
PE_yr_range = 15 # test for 'X' years PE 


# storage process
for j, symbol in enumerate(ticker_symbols):
    print(j, symbol)
    # # COMPANY OVERVIEW
    # # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    # url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={alpha_vantage_api_key}'
    # r = requests.get(url)
    # data = r.json()

    
    # for key, value in data.items():
    #     if key  == 'MarketCapitalization':
    #         stock_mkt_cap = float(value)



    # STOCK SPLIT FACTOR section
    url = f'https://www.alphavantage.co/query?function=SPLITS&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'data':
            if len(value) > 0:
                stock_split_record_df = pd.DataFrame(value)
                stock_split_record_df['split_factor'] = pd.to_numeric(stock_split_record_df['split_factor'], errors='coerce') # change split_factor series to numeric data
                stock_split_record_df['effective_date'] = pd.to_datetime(stock_split_record_df['effective_date'])
            else:
                # Use pandas index assign instead of direct assign value
                stock_split_record_df = pd.DataFrame({
                'split_factor': [1],
                'effective_date': [datetime.today()]
            })


    # Monthly quote section
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'Monthly Time Series':
            Monthly_stock_df = pd.DataFrame(value)


    Monthly_stock_df = Monthly_stock_df.transpose()
    Monthly_stock_df.index = pd.to_datetime(Monthly_stock_df.index)


    filter_1 = (Monthly_stock_df.index.year.isin(range((datetime.today().year - PE_yr_range) ,datetime.today().year)))
    filter_2 = (Monthly_stock_df.index.month == 12) # month = 12 to get the year end closing price

    selected_cols = [
        '4. close'
    ]

    Monthly_stock_df = Monthly_stock_df[
        filter_1
        & filter_2
    ][selected_cols]

    # Rename columns
    Monthly_stock_df.rename(
        columns={
            '4. close': 'stock_price'
            }
        ,inplace=True
        )

    Monthly_stock_df["stock_price"] = Monthly_stock_df["stock_price"].astype(str).apply(lambda x: float(x))
    Monthly_stock_df["stock_price"] = Monthly_stock_df["stock_price"].round(2)

    # modify stock price based on stock split
    for year_i in Monthly_stock_df.index.year:
        for year_j in stock_split_record_df['effective_date'].dt.year:
            if year_i == year_j:

                # stock price to divided the split factor
                Monthly_stock_df.loc[Monthly_stock_df.index.year < year_j, 'stock_price'] /= (stock_split_record_df['split_factor'][stock_split_record_df['effective_date'].dt.year == year_j].values[0])
    


    # Earning section
    # past earnings from alpha vintage API
    url = f'https://www.alphavantage.co/query?function=EARNINGS&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'annualEarnings':

            selected_cols = [
                'fiscalDateEnding'
                ,'reportedEPS'
            ]

            annualEPS_df = pd.DataFrame(value) # tranpose the dataframe and sub select selected cols


            annualEPS_df['fiscalDateEnding'] = pd.to_datetime(annualEPS_df['fiscalDateEnding']).dt.year

            annualEPS_df = annualEPS_df[
                annualEPS_df['fiscalDateEnding'].isin(
                    range(
                        (datetime.today().year - PE_yr_range) 
                        ,datetime.today().year
                            )
                            )
                            ]

            # Convert the column to decimal type
            for col in selected_cols:
                if col in ['reportedEPS']:
                    annualEPS_df[f'{col}'] = annualEPS_df[f'{col}'].astype(str).apply(lambda x: float(x))

                else:
                    continue
            # clean annualEPS_df
            annualEPS_df = annualEPS_df.sort_values('reportedEPS', ascending=False).drop_duplicates('fiscalDateEnding')
            annualEPS_df = annualEPS_df.sort_values('fiscalDateEnding', ascending=False).reset_index(drop=True)

            # calculate PE
            annualEPS_df["PE"] = Monthly_stock_df["stock_price"].values / annualEPS_df['reportedEPS'].values




    # INCOME STATEMENT
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()


    for key, value in data.items():
        if key == 'annualReports':
            annual_income_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            annual_income_df = annual_income_df.sort_values('fiscalDateEnding', ascending=True)

            # annual measurements cols
            annual_income_metric_cols = annual_income_df.columns[1:]

            # Calculate quarter-over-quarter change percentage
            for column in annual_income_metric_cols:  # Exclude the 'fiscalDateEnding' column
                annual_income_df[column] = pd.to_numeric(annual_income_df[column], errors='coerce')
                annual_income_df[f'{column}_YoY'] = annual_income_df[column].pct_change() * 100 
            
            annual_income_YoY_metric_cols = [col for col in annual_income_df.columns if 'YoY' in col]

            # ratios of income statement calculation
            annual_income_df['gross_margin_%'] = (annual_income_df['grossProfit'] / annual_income_df['totalRevenue']) * 100
            annual_income_df['operating_margin_%'] = (annual_income_df['operatingIncome'] / annual_income_df['totalRevenue']) * 100
            annual_income_df['net_margin_%'] = (annual_income_df['netIncome'] / annual_income_df['totalRevenue']) * 100



        if key == 'quarterlyReports':
            qtr_income_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            qtr_income_df = qtr_income_df.sort_values('fiscalDateEnding', ascending=True)

            # qtr measurements cols
            qtr_income_metric_cols = qtr_income_df.columns[1:]
            
            # Calculate quarter-over-quarter change percentage
            for column in qtr_income_df.columns[1:]:  # Exclude the 'fiscalDateEnding' column
                qtr_income_df[column] = pd.to_numeric(qtr_income_df[column], errors='coerce')
                qtr_income_df[f'{column}_QoQ'] = qtr_income_df[column].pct_change() * 100

            qtr_income_QoQ_metric_cols = [col for col in qtr_income_df.columns if 'QoQ' in col]

            # ratios of income statement calculation
            qtr_income_df['gross_margin_%'] = (qtr_income_df['grossProfit'] / qtr_income_df['totalRevenue']) * 100
            qtr_income_df['operating_margin_%'] = (qtr_income_df['operatingIncome'] / qtr_income_df['totalRevenue']) * 100
            qtr_income_df['net_margin_%'] = (qtr_income_df['netIncome'] / qtr_income_df['totalRevenue']) * 100

    income_ratio_cols = [
        'gross_margin_%'
        ,'operating_margin_%'
        ,'net_margin_%'
        ,'netIncome'
        ]
    
    
    # BALANCESHEET
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=BALANCE_SHEET&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():

        if key == 'annualReports':
            annual_balancesheet_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            annual_balancesheet_df = annual_balancesheet_df.sort_values('fiscalDateEnding', ascending=True)

            for column in annual_balancesheet_df.columns[1:]:  # Exclude the 'fiscalDateEnding' column
                annual_balancesheet_df[column] = pd.to_numeric(annual_balancesheet_df[column], errors='coerce')
                annual_balancesheet_df[f'{column}_YoY'] = annual_balancesheet_df[column].pct_change() * 100

            annual_balancesheet_df['current_ratio'] = (annual_balancesheet_df['totalCurrentAssets'] / annual_balancesheet_df['totalCurrentLiabilities'])
            annual_balancesheet_df['working_capital'] = annual_balancesheet_df['totalCurrentAssets'] - annual_balancesheet_df['totalCurrentLiabilities']
            annual_balancesheet_df['longTermDebt_to_workingCp_ratio'] = (annual_balancesheet_df['longTermDebt'] / annual_balancesheet_df['working_capital'])
            annual_balancesheet_df['debtEquity_ratio'] = (annual_balancesheet_df['totalLiabilities'] / annual_balancesheet_df['totalShareholderEquity'])
            annual_balancesheet_df['quick_ratio'] = ((annual_balancesheet_df['totalCurrentAssets'] - annual_balancesheet_df['inventory']) / annual_balancesheet_df['totalCurrentLiabilities'])

            
        if key == 'quarterlyReports':
            qtr_balancesheet_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            qtr_balancesheet_df = qtr_balancesheet_df.sort_values('fiscalDateEnding', ascending=True)

            for column in qtr_balancesheet_df.columns[1:]:  # Exclude the 'fiscalDateEnding' column
                qtr_balancesheet_df[column] = pd.to_numeric(qtr_balancesheet_df[column], errors='coerce')
                qtr_balancesheet_df[f'{column}_YoY'] = qtr_balancesheet_df[column].pct_change() * 100

            qtr_balancesheet_df['current_ratio'] = (qtr_balancesheet_df['totalCurrentAssets'] / qtr_balancesheet_df['totalCurrentLiabilities'])
            qtr_balancesheet_df['working_capital'] = qtr_balancesheet_df['totalCurrentAssets'] - qtr_balancesheet_df['totalCurrentLiabilities']
            qtr_balancesheet_df['longTermDebt_to_workingCp_ratio'] = (qtr_balancesheet_df['longTermDebt'] / qtr_balancesheet_df['working_capital'])
            qtr_balancesheet_df['debtEquity_ratio'] = (qtr_balancesheet_df['totalLiabilities'] / qtr_balancesheet_df['totalShareholderEquity'])
            qtr_balancesheet_df['quick_ratio'] = ((qtr_balancesheet_df['totalCurrentAssets'] - qtr_balancesheet_df['inventory']) / qtr_balancesheet_df['totalCurrentLiabilities'])


    balancesheet_ratio_cols = [
        'current_ratio'
        ,'working_capital'
        ,'longTermDebt_to_workingCp_ratio'
        ,'debtEquity_ratio'
        ,'quick_ratio'
        ,'totalShareholderEquity'
    ]


    # CASHFLOW STATEMENT
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=CASH_FLOW&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():

        if key == 'annualReports':
            annual_cashflow_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            annual_cashflow_df = annual_cashflow_df.sort_values('fiscalDateEnding', ascending=True)

            for column in annual_cashflow_df.columns[1:]:  # Exclude the 'fiscalDateEnding' column
                annual_cashflow_df[column] = pd.to_numeric(annual_cashflow_df[column], errors='coerce')
                annual_cashflow_df[f'{column}_YoY'] = annual_cashflow_df[column].pct_change() * 100

            
        if key == 'quarterlyReports':
            qtr_cashflow_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            qtr_cashflow_df = qtr_cashflow_df.sort_values('fiscalDateEnding', ascending=True)

            for column in qtr_cashflow_df.columns[1:]:  # Exclude the 'fiscalDateEnding' column
                qtr_cashflow_df[column] = pd.to_numeric(qtr_cashflow_df[column], errors='coerce')
                qtr_cashflow_df[f'{column}_YoY'] = qtr_cashflow_df[column].pct_change() * 100



    cashflow_ratio_cols = [
        'operatingCashflow'
        ,'capitalExpenditures'
        ,'cashflowFromInvestment'
        ,'cashflowFromFinancing'
        ,'netIncome'
    ]


    ####################
    ### Consolidated ###
    ####################
    stock_ratios_annual_consolidate_df = pd.DataFrame()
    stock_ratios_qtr_consolidate_df = pd.DataFrame()
    stock_PE_annual_consolidate_df = pd.DataFrame()

    stock_ratios_annual_consolidate_df[income_ratio_cols] = annual_income_df[income_ratio_cols]
    stock_ratios_annual_consolidate_df[balancesheet_ratio_cols] = annual_balancesheet_df[balancesheet_ratio_cols]
    stock_ratios_annual_consolidate_df[cashflow_ratio_cols] = annual_cashflow_df[cashflow_ratio_cols]

    stock_ratios_qtr_consolidate_df[income_ratio_cols] = qtr_income_df[income_ratio_cols]
    stock_ratios_qtr_consolidate_df[balancesheet_ratio_cols] = qtr_balancesheet_df[balancesheet_ratio_cols]
    stock_ratios_qtr_consolidate_df[cashflow_ratio_cols] = qtr_cashflow_df[cashflow_ratio_cols]

    # calculating new consolidated metrics 
    # annual df
    stock_ratios_annual_consolidate_df['ROE_%'] = (annual_income_df['netIncome'] / annual_balancesheet_df['totalShareholderEquity']) * 100
    stock_ratios_annual_consolidate_df['liquidation_value'] = ((annual_balancesheet_df['totalAssets'] - annual_balancesheet_df['intangibleAssets']) - annual_balancesheet_df['totalLiabilities'])
    # stock_ratios_annual_consolidate_df['liquidation_mktcap_ratio_%'] = round((stock_ratios_annual_consolidate_df['liquidation_value'] / stock_mkt_cap) * 100, 2)
    stock_ratios_annual_consolidate_df['fiscalDateEnding'] = annual_balancesheet_df['fiscalDateEnding']

    stock_ratios_annual_consolidate_df['PE'] = annualEPS_df["PE"]
    stock_ratios_annual_consolidate_df['reportedEPS'] = annualEPS_df["reportedEPS"]

    # YoY %
    stock_ratios_annual_consolidate_df['QoQ_gross_margin_%_%'] = stock_ratios_annual_consolidate_df["gross_margin_%"].pct_change() * 100
    stock_ratios_annual_consolidate_df['QoQ_operating_margin_%_%'] = stock_ratios_annual_consolidate_df["operating_margin_%"].pct_change() * 100
    stock_ratios_annual_consolidate_df['QoQ_net_margin_%_%'] = stock_ratios_annual_consolidate_df["net_margin_%"].pct_change() * 100
    stock_ratios_annual_consolidate_df['YoY_reportedEPS_%'] = stock_ratios_annual_consolidate_df["reportedEPS"].pct_change() * 100 # yearly EPS 的增速



    # qtr df
    stock_ratios_qtr_consolidate_df['ROE_%'] = (qtr_income_df['netIncome'] / qtr_balancesheet_df['totalShareholderEquity']) * 100
    stock_ratios_qtr_consolidate_df['liquidation_value'] = ((qtr_balancesheet_df['totalAssets'] - qtr_balancesheet_df['intangibleAssets']) - qtr_balancesheet_df['totalLiabilities'])
    # stock_ratios_qtr_consolidate_df['liquidation_mktcap_ratio_%'] = round((stock_ratios_qtr_consolidate_df['liquidation_value'] / stock_mkt_cap) * 100, 2)
    stock_ratios_qtr_consolidate_df['fiscalDateEnding'] = qtr_balancesheet_df['fiscalDateEnding']
    
    # QoQ %
    stock_ratios_qtr_consolidate_df['QoQ_gross_margin_%_%'] = stock_ratios_qtr_consolidate_df["gross_margin_%"].pct_change() * 100
    stock_ratios_qtr_consolidate_df['QoQ_operating_margin_%_%'] = stock_ratios_qtr_consolidate_df["operating_margin_%"].pct_change() * 100
    stock_ratios_qtr_consolidate_df['QoQ_net_margin_%_%'] = stock_ratios_qtr_consolidate_df["net_margin_%"].pct_change() * 100




    # annual PE df
    if stock_PE_annual_consolidate_df.empty:
        stock_PE_annual_consolidate_df['fiscalDateEnding'] = annualEPS_df['fiscalDateEnding']

    stock_PE_annual_consolidate_df[f'{symbol}_PE'] = annualEPS_df["PE"].round(2)



    # reformat the columns order
    stock_ratios_annual_consolidate_df = stock_ratios_annual_consolidate_df[['fiscalDateEnding'] + [col for col in stock_ratios_annual_consolidate_df.columns if col != 'fiscalDateEnding']]
    stock_ratios_qtr_consolidate_df = stock_ratios_qtr_consolidate_df[['fiscalDateEnding'] + [col for col in stock_ratios_qtr_consolidate_df.columns if col != 'fiscalDateEnding']]




    # store the stock, dataframe value pair to the dictionary
    # transfer the pandas df to json
    stock_ratios_annual_consolidate_json = stock_ratios_annual_consolidate_df.to_dict()
    stock_ratios_qtr_consolidate_json = stock_ratios_qtr_consolidate_df.to_dict()

    stock_financial_dict[symbol] = {
    'annual': stock_ratios_annual_consolidate_json
    ,'qtr': stock_ratios_qtr_consolidate_json
    }

0 NFLX


In [64]:
qtr_balancesheet_df.tail(10)

,fiscalDateEnding,totalAssets,totalCurrentAssets,cashAndCashEquivalentsAtCarryingValue,cashAndShortTermInvestments,inventory,currentNetReceivables,totalNonCurrentAssets,propertyPlantEquipment,accumulatedDepreciationAmortizationPPE,intangibleAssets,intangibleAssetsExcludingGoodwill,goodwill,investments,longTermInvestments,shortTermInvestments,otherCurrentAssets,otherNonCurrentAssets,totalLiabilities,totalCurrentLiabilities,currentAccountsPayable,deferredRevenue,currentDebt,shortTermDebt,totalNonCurrentLiabilities,capitalLeaseObligations,longTermDebt,currentLongTermDebt,longTermDebtNoncurrent,shortLongTermDebtTotal,otherCurrentLiabilities,otherNonCurrentLiabilities,totalShareholderEquity,treasuryStock,retainedEarnings,commonStock,commonStockSharesOutstanding,totalAssets_YoY,totalCurrentAssets_YoY,cashAndCashEquivalentsAtCarryingValue_YoY,cashAndShortTermInvestments_YoY,inventory_YoY,currentNetReceivables_YoY,totalNonCurrentAssets_YoY,propertyPlantEquipment_YoY,accumulatedDepreciationAmortizationPPE_YoY,intangibleAssets_YoY,intangibleAssetsExcludingGoodwill_YoY,goodwill_YoY,investments_YoY,longTermInvestments_YoY,shortTermInvestments_YoY,otherCurrentAssets_YoY,otherNonCurrentAssets_YoY,totalLiabilities_YoY,totalCurrentLiabilities_YoY,currentAccountsPayable_YoY,deferredRevenue_YoY,currentDebt_YoY,shortTermDebt_YoY,totalNonCurrentLiabilities_YoY,capitalLeaseObligations_YoY,longTermDebt_YoY,currentLongTermDebt_YoY,longTermDebtNoncurrent_YoY,shortLongTermDebtTotal_YoY,otherCurrentLiabilities_YoY,otherNonCurrentLiabilities_YoY,totalShareholderEquity_YoY,treasuryStock_YoY,retainedEarnings_YoY,commonStock_YoY,commonStockSharesOutstanding_YoY,current_ratio,working_capital,longTermDebt_to_workingCp_ratio,debtEquity_ratio,quick_ratio
9,2023-09-30,49501786000,10779474000,7353245000,7353245000,4.823750e+08,1.139974e+09,38722312000,3.622892e+09,NaN,3.174986e+10,3.174986e+10,NaN,NaN,NaN,5.142010e+08,1772054000,NaN,27394159000,8338718000,534429000,NaN,NaN,7.770140e+08,19055441000,NaN,1.390075e+10,3.996140e+08,NaN,1.676425e+10,5721090000,5.154687e+09,22107627000,NaN,21651448000,5011427000,4500110000,-2.589045,-6.317059,-4.039561,-4.039561,-1.492688e+02,-6.431119,-1.497858,146.125731,NaN,-2.370525,-2.370525,NaN,NaN,0.0,-43.754054,-39.506859,NaN,-2.112180,-3.885369,-13.153789,NaN,NaN,94.551650,-1.315480,0.0,-1.203946,0.056837,NaN,15.858903,-9.891692,-1.615005,-3.173534,0.0,8.398017,2.815206,-0.345681,1.292702,2440756000,5.695266,1.239127,1.234854
8,2023-12-31,48731992000,9918133000,7116913000,7116913000,4.089360e+08,1.842054e+09,38813859000,NaN,NaN,3.165806e+10,3.165806e+10,NaN,NaN,NaN,2.097300e+07,938193000,NaN,28143679000,8860655000,747412000,NaN,NaN,7.831560e+08,19283024000,NaN,1.414342e+10,3.998440e+08,NaN,1.697337e+10,5887118000,5.139607e+09,20588313000,NaN,22589286000,5145172000,4494980000,-1.555083,-7.990566,-3.213982,-3.213982,-1.522446e+01,61.587370,0.236419,0.000000,NaN,-0.289151,-0.289151,NaN,NaN,0.0,-95.921245,-47.056185,NaN,2.736058,6.259200,39.852441,NaN,NaN,0.790462,1.194320,0.0,1.745682,0.057556,NaN,1.247452,2.902034,-0.292549,-6.872352,0.0,4.331526,2.668801,-0.113997,1.119345,1057478000,13.374668,1.366974,1.073193
7,2024-03-31,48827721000,9921313000,7024766000,7024766000,-3.166210e+10,1.228691e+09,38906408000,NaN,NaN,3.166210e+10,3.166210e+10,NaN,NaN,NaN,2.097300e+07,1646883000,NaN,27462311000,9289217000,607348000,NaN,NaN,1.196564e+09,18173094000,NaN,1.321704e+10,7.989360e+08,NaN,1.651161e+10,6015821000,4.956056e+09,21365410000,NaN,24921495000,5489850000,4416540000,0.196440,0.032062,-1.294761,-1.294761,-7.842556e+03,-33.297775,0.238443,0.000000,NaN,0.012774,0.012774,NaN,NaN,0.0,0.000000,75.537762,NaN,-2.421034,4.836685,-18.739865,NaN,NaN,52.787440,-5.755996,0.0,-6.549895,99.811927,NaN,-2.720543,2.186180,-3.571304,3.774457,0.0,10.324403,6.699057,-1.745058,1.068046,632096000,20.909859,1.285363,4.476525
6,2024-06-30,49098895000,9615553000,6624939000,6624939000,1.000000e+00,1.276359e+09,39483342000,NaN,NaN,3.192736e+10,3.192736e+10,Na

In [65]:
qtr_income_df.tail(20)

,fiscalDateEnding,grossProfit,totalRevenue,costOfRevenue,costofGoodsAndServicesSold,operatingIncome,sellingGeneralAndAdministrative,researchAndDevelopment,operatingExpenses,investmentIncomeNet,netInterestIncome,interestIncome,interestExpense,nonInterestIncome,otherNonOperatingIncome,depreciation,depreciationAndAmortization,incomeBeforeTax,incomeTaxExpense,interestAndDebtExpense,netIncomeFromContinuingOperations,comprehensiveIncomeNetOfTax,ebit,ebitda,netIncome,grossProfit_QoQ,totalRevenue_QoQ,costOfRevenue_QoQ,costofGoodsAndServicesSold_QoQ,operatingIncome_QoQ,sellingGeneralAndAdministrative_QoQ,researchAndDevelopment_QoQ,operatingExpenses_QoQ,investmentIncomeNet_QoQ,netInterestIncome_QoQ,interestIncome_QoQ,interestExpense_QoQ,nonInterestIncome_QoQ,otherNonOperatingIncome_QoQ,depreciation_QoQ,depreciationAndAmortization_QoQ,incomeBeforeTax_QoQ,incomeTaxExpense_QoQ,interestAndDebtExpense_QoQ,netIncomeFromContinuingOperations_QoQ,comprehensiveIncomeNetOfTax_QoQ,ebit_QoQ,ebitda_QoQ,netIncome_QoQ,gross_margin_%,operating_margin_%,net_margin_%
19,2021-03-31,3294771000,7163282000,3868511000,3868511000,1959856000,297196000,525207000,1334915000,NaN,74646000.0,269086000.0,194440000,NaN,269086000.0,NaN,2.754937e+09,2034502000,327787000.0,NaN,1706715000,NaN,1.959856e+09,4.714793e+09,1706715000,32.892144,7.808632,-7.122151,-7.122151,105.383540,7.859867,7.859554,-12.466886,NaN,-116.668565,102.054440,-56.581254,NaN,-207.359988,NaN,-8.612544,301.744412,-1017.168919,NaN,214.801459,NaN,105.383540,18.796138,214.801459,45.995271,27.359749,23.825880
18,2021-06-30,3323769000,7341777000,4018008000,4018008000,1847630000,334845000,537321000,1476139000,NaN,-253841000.0,NaN,253841000,NaN,-62519000.0,NaN,2.845237e+09,1593789000,240776000.0,NaN,1353013000,NaN,1.847630e+09,4.692867e+09,1353013000,0.880122,2.491805,3.864458,3.864458,-5.726237,12.668071,2.306519,10.579250,NaN,-440.059749,0.000000,30.549784,NaN,-123.233836,NaN,3.277752,-21.661960,-26.544982,NaN,-20.724140,NaN,-5.726237,-0.465047,-20.724140,45.271996,25.165978,18.428958
17,2021-09-30,3276878000,7483467000,4206589000,4206589000,1755253000,321790000,563887000,1521625000,NaN,-94294000.0,96135000.0,190429000,NaN,96135000.0,NaN,3.033304e+09,1660959000,211888000.0,NaN,1449071000,NaN,1.755253e+09,4.788557e+09,1449071000,-1.410778,1.929914,4.693395,4.693395,-4.999756,-3.898819,4.944158,3.081417,NaN,-62.853125,-64.273504,-24.980992,NaN,-253.769254,NaN,6.609889,4.214485,-11.997874,NaN,7.099562,NaN,-4.999756,2.039052,7.099562,43.788233,23.455078,19.363632
16,2021-12-31,2469743000,7709318000,5239575000,5239575000,631770000,397790000,647470000,1837973000,NaN,-80917000.0,108512000.0,189429000,NaN,108512000.0,NaN,3.805301e+09,550853000,-56576000.0,NaN,607429000,NaN,6.317700e+08,4.437071e+09,607429000,-24.631219,3.018000,24.556380,24.556380,-64.006898,23.617887,14.822651,20.790142,NaN,-14.186481,12.874603,-0.525130,NaN,12.874603,NaN,25.450697,-66.835244,-126.700899,NaN,-58.081488,NaN,-64.006898,-7.340124,-58.081488,32.035817,8.194888,7.879154
15,2022-03-31,3583062000,7867767000,4284705000,4284705000,1971626000,397928000,657530000,1611436000,NaN,8066000.0,195645000.0,187579000,NaN,195645000.0,NaN,3.436612e+09,1979692000,382245000.0,NaN,1597447000,NaN,1.971626e+09,5.408238e+09,1597447000,45.078334,2.055292,-18.224188,-18.224188,212.079713,0.034692,1.553740,-12.325371,NaN,-109.968239,80.298032,-0.976619,NaN,80.298032,NaN,-9.688826,259.386624,-775.631010,NaN,162.984974,NaN,212.079713,21.887570,162.984974,45.541028,25.059537,20.303690
14,2022-06-30,3279386000,7970141000,4690755000,4690755000,1578283000,409297000,716846000,1701103000,NaN,44771000.0,220226000.0,175455000,NaN,220226000.0,NaN,3.565079e+09,1623054000,182103000.0,NaN,1440951000,NaN,1.578283e+09,5.143362e+09,1440951000,-8.475321,1.301182,9.476732,9.476732,-19.950183,2.857050,9.021033,5.564416,NaN,455.058269,12.564083,-6.463410,NaN,12.564083,NaN,3.738187,-18.014823,-52.359612,NaN,-9.796632,NaN,-19.950183,-4.897639,-9.796632,41.145897,19.802448,18.079366


In [66]:
pd.DataFrame(stock_financial_dict['NFLX']['qtr']).tail(20)

,fiscalDateEnding,gross_margin_%,operating_margin_%,net_margin_%,netIncome,current_ratio,working_capital,longTermDebt_to_workingCp_ratio,debtEquity_ratio,quick_ratio,totalShareholderEquity,operatingCashflow,capitalExpenditures,cashflowFromInvestment,cashflowFromFinancing,ROE_%,liquidation_value,QoQ_gross_margin_%_%,QoQ_operating_margin_%_%,QoQ_net_margin_%_%
19,2021-03-31,45.995271,27.359749,23.825880,1706715000,1.269505,2145738000,6.925613,2.114154,NaN,12884080000,777266000,81001000,-85616000,-451929000,13.246697,-1.315991e+10,23.266701,90.507510,192.000237
18,2021-06-30,45.271996,25.165978,18.428958,1353013000,1.226147,1771388000,8.426663,1.955233,NaN,13863871000,-63761000,110278000,-111278000,-480273000,9.759273,-1.342777e+10,-1.572499,-8.018243,-22.651513
17,2021-09-30,43.788233,23.455078,19.363632,1449071000,1.169773,1366541000,10.825647,1.790788,NaN,15314617000,82379000,167327000,-188631000,-81555000,9.462013,-1.365943e+10,-3.277440,-6.798466,5.071767
16,2021-12-31,32.035817,8.194888,7.879154,607429000,0.950625,-419141000,-35.055201,1.813046,NaN,15849248000,-403274000,165979000,-954328000,-136019000,3.832541,-1.507029e+10,-26.839211,-65.061347,-59.309526
15,2022-03-31,45.541028,25.059537,20.303690,1597447000,1.046302,358359000,40.558660,1.583835,NaN,17544039000,922839000,121158000,-245679000,-686322000,9.105355,-1.364788e+10,42.156601,205.794738,157.688719
14,2022-06-30,41.145897,19.802448,18.079366,1440951000,1.045434,340756000,41.769780,1.429807,NaN,19075974000,102750000,90018000,-158894000,11250000,7.553748,-1.345722e+10,-9.650927,-20.978398,-10.955266
13,2022-09-30,39.579696,19.342638,17.642121,1398242000,1.135332,1050979000,13.214457,1.316926,NaN,20528141000,556810000,84960000,-84960000,4113000,6.811343,-1.224920e+10,-3.806458,-2.321983,-2.418478
12,2022-12-31,31.175197,7.003315,0.704071,55284000,1.168390,1335499000,10.747351,1.338828,NaN,20777401000,443858000,111593000,-1586859000,6705000,0.266078,-1.195931e+10,-21.234369,-63.793383,-96.009150
11,2023-03-31,41.142888,21.004918,15.991172,1305120000,1.260526,2166553000,6.479401,1.267267,NaN,21828196000,2178740000,62019000,-263653000,-374073000,5.979056,-1.052099e+10,31.973143,199.928221,2171.245419
10,2023-06-30,42.918063,22.317281,18.169724,1487610000,1.326256,2830531000,4.970852,1.225692,NaN,22832215000,1440232000,100972000,97737000,-649349000,6.515399,-9.688559e+09,4.314659,6.247887,13.623466


In [67]:
# searching & merge process
YoY_EPS_screener = pd.DataFrame()

for symbol in stock_financial_dict.keys():
    annaul_df = pd.DataFrame(
        stock_financial_dict[symbol]['annual']
    )

    annaul_df = annaul_df[['fiscalDateEnding', 'YoY_reportedEPS_%', 'reportedEPS']].rename(
        columns={
            'YoY_reportedEPS_%': f'{symbol}_YoY_reportedEPS_%'
            ,'reportedEPS': f'{symbol}_reportedEPS'
            }
    )
    annaul_df['fiscalDateEnding'] = pd.to_datetime(annaul_df['fiscalDateEnding']).dt.year


    if YoY_EPS_screener.empty:
        YoY_EPS_screener = annaul_df
    else:
        YoY_EPS_screener = pd.merge(
            YoY_EPS_screener
            ,annaul_df
            ,on='fiscalDateEnding'
            ,how='outer'
        )


In [68]:
YoY_EPS_screener

,fiscalDateEnding,NFLX_YoY_reportedEPS_%,NFLX_reportedEPS
19,2006,NaN,NaN
18,2007,NaN,NaN
17,2008,NaN,NaN
16,2009,NaN,NaN
15,2010,NaN,NaN
14,2011,NaN,0.07
13,2012,-28.571429,0.05
12,2013,40.000000,0.07
11,2014,-28.571429,0.05
10,2015,-20.000000,0.04


In [69]:
stock_PE_annual_consolidate_df

,fiscalDateEnding,NFLX_PE
0,2025,37.06
1,2024,45.02
2,2023,40.57
3,2022,29.79
4,2021,53.31
5,2020,88.64
6,2019,77.04
7,2018,99.13
8,2017,147.66
9,2016,247.60


# SP500 sectors

## SP500 sector ticker

In [ ]:
# Function to get the list of S&P 500 companies and their sectors
def get_sp500_companies():
    # Fetch the S&P 500 company symbols and sectors from a reliable source (e.g., Wikipedia)
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    tables = pd.read_html(url)
    
    # Extract the relevant table containing the company symbols and sectors
    sp500_df = tables[0]
    
    # Return the DataFrame containing S&P 500 companies and sectors
    return sp500_df[['Symbol', 'GICS Sector', 'GICS Sub-Industry']]


# Function to create a dictionary of sectors and sub-sectors
def create_sector_subsector_dict(df):
    sector_subsector_dict = {}
    for index, row in df.iterrows():
        sector = row['GICS Sector']
        subsector = row['GICS Sub-Industry']
        if sector not in sector_subsector_dict:
            sector_subsector_dict[sector] = [subsector]
        else:
            sector_subsector_dict[sector].append(subsector)
    return sector_subsector_dict

# Function to filter the S&P 500 companies by sector
def company_sector_list(df, sector):
    return df[df['GICS Sector'] == sector]['Symbol'].tolist()

def company_sub_sector_list(df, sub_sector):
    return df[df['GICS Sub-Industry'] == sub_sector]['Symbol'].tolist()


# Get the list of S&P 500 companies and their sectors
sp500_df  = get_sp500_companies()

sp500_companies_sectors = sp500_df ['GICS Sector'].value_counts().index
sp500_companies_sub_sectors = sp500_df ['GICS Sub-Industry'].value_counts().index

sector_subsector_dict = create_sector_subsector_dict(sp500_df)


# Function to create a DataFrame from the sector_subsector_dict
def create_sector_dataframe():
    # Create a list to store dictionacompany_sector_listries representing each row of data
    data = []
    
    # Filter the DataFrame to get stocks in the specified sector
    for sector in sp500_companies_sectors:
        sector_stocks_list = company_sector_list(sp500_df, sector)

        # Iterate over the stocks in the sector and create a dictionary for each
        for i, ticker in enumerate(sector_stocks_list, start=1):
            # Create a dictionary for the current stock in the sector
            row_data = {'Sector': sector, 'Ticker': ticker}
            # Append the dictionary to the list
            data.append(row_data)
    
    # Create a DataFrame from the list of dictionaries
    df = pd.DataFrame(data)
    return df


sector_subsector_dict = create_sector_subsector_dict(sp500_df)


sector_ticker_df = create_sector_dataframe()
# sector_ticker_list = sector_ticker_df[sector_ticker_df['Sector'] == 'Consumer Staples']['Ticker'].tolist()

NameError: name 'pd' is not defined

In [ ]:
# Pivot the DataFrame
sp500_pivot_sector_ticker_df = sector_ticker_df.groupby('Sector')['Ticker'].apply(list).reset_index()

# Transpose to get sectors as columns
sp500_pivot_sector_ticker_df = sp500_pivot_sector_ticker_df.set_index('Sector').T

sp500_pivot_sector_ticker_df

Sector,Communication Services,Consumer Discretionary,Consumer Staples,Energy,Financials,Health Care,Industrials,Information Technology,Materials,Real Estate,Utilities
Ticker,"[GOOGL, GOOG, T, CHTR, CMCSA, EA, FOXA, FOX, I...","[ABNB, AMZN, APTV, AZO, BBY, BKNG, BWA, CZR, K...","[MO, ADM, BF.B, BG, CPB, CHD, CLX, KO, CL, CAG...","[APA, BKR, CVX, COP, CTRA, DVN, FANG, EOG, EQT...","[AFL, ALL, AXP, AIG, AMP, AON, APO, ACGL, AJG,...","[ABT, ABBV, A, ALGN, AMGN, BAX, BDX, TECH, BII...","[MMM, AOS, ALLE, AME, ADP, AXON, BA, BR, BLDR,...","[ACN, ADBE, AMD, AKAM, APH, ADI, ANSS, AAPL, A...","[APD, ALB, AMCR, AVY, BALL, CE, CF, CTVA, DOW,...","[ARE, AMT, AVB, BXP, CPT, CBRE, CSGP, CCI, DLR...","[AES, LNT, AEE, AEP, AWK, ATO, CNP, CMS, ED, C..."


In [ ]:
sp500_pivot_sector_ticker_df['Consumer Staples'].iloc[0]

['MO',
 'ADM',
 'BF.B',
 'BG',
 'CPB',
 'CHD',
 'CLX',
 'KO',
 'CL',
 'CAG',
 'STZ',
 'COST',
 'DG',
 'DLTR',
 'EL',
 'GIS',
 'HSY',
 'HRL',
 'K',
 'KVUE',
 'KDP',
 'KMB',
 'KHC',
 'KR',
 'LW',
 'MKC',
 'TAP',
 'MDLZ',
 'MNST',
 'PEP',
 'PM',
 'PG',
 'SJM',
 'SYY',
 'TGT',
 'TSN',
 'WBA',
 'WMT']

In [ ]:
len(sp500_pivot_sector_ticker_df['Consumer Discretionary'].iloc[0])

50

## SP 500 earnings by years

In [ ]:
sp500_eps_df= pd.read_excel('D:\Investment\SP500_sectors\sp500_earnings\sp-500-eps-est.xlsx'
                            ,sheet_name='QUARTERLY DATA')

In [ ]:
sp500_eps_df.columns = [
    'Quarter'
    ,'Ops EPS(Opearting)'
    ,'Reported EPS(GAAP)'
    ,'Div/share'
    ,'Sales/share'
    ,'BVPS'
    ,'CapEx/share'
    ,'Price'
    ,'Divisor'
]

In [ ]:
sp500_eps_df = sp500_eps_df[5::].fillna(0)
sp500_eps_df_no_first = sp500_eps_df \
    .iloc[1:] \
    .sort_values(['Quarter'], ascending=True) \
    .copy()

#####################
# Column enrichment #
#####################
sp500_eps_df['Year'] = pd.to_datetime(sp500_eps_df['Quarter']).dt.year
sp500_eps_df["Month"] = pd.to_datetime(sp500_eps_df['Quarter']).dt.month

# Calculate year end EPS
last_quarter_prices_df = sp500_eps_df[sp500_eps_df["Month"] == 12][["Year", "Price"]]
price_map = dict(zip(last_quarter_prices_df["Year"], last_quarter_prices_df["Price"]))
sp500_eps_df["Yearly price"] = sp500_eps_df["Year"].map(price_map)


# Calculate rolling 4-quarter sum
sp500_eps_df_no_first["Ops EPS(Opearting) TTM"] = sp500_eps_df_no_first["Ops EPS(Opearting)"].rolling(window=4).sum()
sp500_eps_df_no_first["Reported EPS(GAAP) TTM"] = sp500_eps_df_no_first["Reported EPS(GAAP)"].rolling(window=4).sum()
sp500_eps_df_no_first["Div/share TTM"] = sp500_eps_df_no_first["Div/share"].rolling(window=4).sum()


# Merge back with original dataframe
sp500_eps_df = sp500_eps_df.merge(
    sp500_eps_df_no_first[
        [
            "Quarter"
            ,"Ops EPS(Opearting) TTM"
            ,'Reported EPS(GAAP) TTM'
            ,'Div/share TTM'
            ]
        ]
    ,on="Quarter"
    ,how="left"
    )


######################
# Metrics calcuation #
######################
sp500_eps_df['PE TTM Ops'] = sp500_eps_df['Price'] / sp500_eps_df['Ops EPS(Opearting) TTM']
sp500_eps_df['PE TTM GAAP'] = sp500_eps_df['Price'] / sp500_eps_df['Reported EPS(GAAP) TTM']
sp500_eps_df['Earnings yield(Ops)'] = round((sp500_eps_df['Ops EPS(Opearting) TTM'] / sp500_eps_df['Price']) * 100, 2)
sp500_eps_df['Earnings yield(GAAP)'] = round((sp500_eps_df['Reported EPS(GAAP) TTM'] / sp500_eps_df['Price']) * 100, 2)
sp500_eps_df['Div payout ratio(Opearting)'] = round((sp500_eps_df['Div/share TTM'] / sp500_eps_df['Ops EPS(Opearting) TTM']) * 100 ,2)
sp500_eps_df['Div payout ratio(GAAP)'] = round((sp500_eps_df['Div/share TTM'] / sp500_eps_df['Reported EPS(GAAP) TTM']) * 100, 2)
# sp500_eps_df["Ops EPS(Opearting) TTM Growth Rate"] = round(((sp500_eps_df["Ops EPS(Opearting) TTM"] - sp500_eps_df["Ops EPS(Opearting) TTM"].shift(-1)) 
#                                                        / sp500_eps_df["Ops EPS(Opearting) TTM"])*100, 2)
# sp500_eps_df["Reported EPS(GAAP) TTM Growth Rate"] = round(((sp500_eps_df["Reported EPS(GAAP) TTM"] - sp500_eps_df["Reported EPS(GAAP) TTM"].shift(-1)) 
#                                                        / sp500_eps_df["Reported EPS(GAAP) TTM"])*100, 2)


# 1.section: YoY growth rate
sp500_eps_df_yrly_ops_eps = sp500_eps_df.groupby(['Year'])['Ops EPS(Opearting)'].sum().reset_index()
sp500_eps_df_yrly_gaap_eps = sp500_eps_df.groupby(['Year'])['Reported EPS(GAAP)'].sum().reset_index()
sp500_eps_df_yrly_ops_eps['YoY growth rate(Ops)'] = sp500_eps_df_yrly_ops_eps['Ops EPS(Opearting)'].pct_change()
sp500_eps_df_yrly_gaap_eps['YoY growth rate(GAAP)'] = sp500_eps_df_yrly_gaap_eps['Reported EPS(GAAP)'].pct_change()

# Create mapping dictionary: {year: YoY growth}
growth_map_ops_eps = dict(zip(sp500_eps_df_yrly_ops_eps["Year"], sp500_eps_df_yrly_ops_eps["YoY growth rate(Ops)"]))
growth_map_gaap_eps = dict(zip(sp500_eps_df_yrly_gaap_eps["Year"], sp500_eps_df_yrly_gaap_eps["YoY growth rate(GAAP)"]))
yrly_map_ops_eps = dict(zip(sp500_eps_df_yrly_ops_eps["Year"], sp500_eps_df_yrly_ops_eps["Ops EPS(Opearting)"]))
yrly_map_gaap_eps = dict(zip(sp500_eps_df_yrly_gaap_eps["Year"], sp500_eps_df_yrly_gaap_eps["Reported EPS(GAAP)"]))

# Map back to original df
sp500_eps_df["YoY growth rate(Ops)"] = round(sp500_eps_df["Year"].map(growth_map_ops_eps)*100, 2)
sp500_eps_df["YoY growth rate(GAAP)"] = round(sp500_eps_df["Year"].map(growth_map_gaap_eps)*100, 2)
sp500_eps_df["Yearly EPS (Ops)"] = sp500_eps_df["Year"].map(yrly_map_ops_eps)
sp500_eps_df["Yearly EPS (GAAP)"] = sp500_eps_df["Year"].map(yrly_map_gaap_eps)


# 2.section: PE TTM AVG Section
PE_TTM_ops_avg = sp500_eps_df.loc[sp500_eps_df["PE TTM Ops"] != 0, "PE TTM Ops"].mean(skipna=True)
PE_TTM_gaap_avg = sp500_eps_df.loc[sp500_eps_df["PE TTM GAAP"] != 0, "PE TTM GAAP"].mean(skipna=True)
sp500_eps_df['PE TTM Ops AVG'] = round(PE_TTM_ops_avg, 2)
sp500_eps_df['PE TTM GAAP AVG'] = round(PE_TTM_gaap_avg, 2)

# 3. section: PE 5 years rolling average
sp500_eps_df['PE Yrly Ops'] = sp500_eps_df['Yearly price'] / sp500_eps_df["Yearly EPS (Ops)"]
sp500_eps_df['PE Yrly GAAP'] = sp500_eps_df["Yearly price"] / sp500_eps_df["Yearly EPS (GAAP)"]

sp500_eps_df_yrly_ops_PE = sp500_eps_df[sp500_eps_df["PE Yrly Ops"] != 0] \
                            .groupby("Year", as_index=False)["PE Yrly Ops"] \
                            .mean()
sp500_eps_df_yrly_gaap_PE = sp500_eps_df[sp500_eps_df["PE Yrly GAAP"] != 0] \
                            .groupby("Year", as_index=False)["PE Yrly GAAP"] \
                            .mean()

sp500_eps_df_yrly_ops_PE["PE Ops 5Y Avg"] = sp500_eps_df_yrly_ops_PE["PE Yrly Ops"].rolling(window=5, min_periods=1).mean()
sp500_eps_df_yrly_ops_PE["PE Ops 10Y Avg"] = sp500_eps_df_yrly_ops_PE["PE Yrly Ops"].rolling(window=10, min_periods=1).mean()
sp500_eps_df_yrly_gaap_PE["PE GAAP 5Y Avg"] = sp500_eps_df_yrly_gaap_PE["PE Yrly GAAP"].rolling(window=5, min_periods=1).mean()
sp500_eps_df_yrly_gaap_PE["PE GAAP 10Y Avg"] = sp500_eps_df_yrly_gaap_PE["PE Yrly GAAP"].rolling(window=10, min_periods=1).mean()


# Create mapping dictionaries
PE5_ops_map = dict(zip(sp500_eps_df_yrly_ops_PE["Year"], sp500_eps_df_yrly_ops_PE["PE Ops 5Y Avg"]))
PE10_ops_map = dict(zip(sp500_eps_df_yrly_ops_PE["Year"], sp500_eps_df_yrly_ops_PE["PE Ops 10Y Avg"]))
PE5_gaap_map = dict(zip(sp500_eps_df_yrly_gaap_PE["Year"], sp500_eps_df_yrly_gaap_PE["PE GAAP 5Y Avg"]))
PE10_gaap_map = dict(zip(sp500_eps_df_yrly_gaap_PE["Year"], sp500_eps_df_yrly_gaap_PE["PE GAAP 10Y Avg"]))


# Map to quarterly dataframe
sp500_eps_df["PE Ops 5Y Avg"] = sp500_eps_df["Year"].map(PE5_ops_map)
sp500_eps_df["PE Ops 10Y Avg"] = sp500_eps_df["Year"].map(PE10_ops_map)
sp500_eps_df["PE GAAP 5Y Avg"] = sp500_eps_df["Year"].map(PE5_gaap_map)
sp500_eps_df["PE GAAP 10Y Avg"] = sp500_eps_df["Year"].map(PE10_gaap_map)

In [ ]:
sp500_eps_df.head(10)

,Quarter,Ops EPS(Opearting),Reported EPS(GAAP),Div/share,Sales/share,BVPS,CapEx/share,Price,Divisor,Year,Month,Yearly price,Ops EPS(Opearting) TTM,Reported EPS(GAAP) TTM,Div/share TTM,PE TTM Ops,PE TTM GAAP,Earnings yield(Ops),Earnings yield(GAAP),Div payout ratio(Opearting),Div payout ratio(GAAP),YoY growth rate(Ops),YoY growth rate(GAAP),Yearly EPS (Ops),Yearly EPS (GAAP),PE TTM Ops AVG,PE TTM GAAP AVG,PE Yrly Ops,PE Yrly GAAP,PE Ops 5Y Avg,PE Ops 10Y Avg,PE GAAP 5Y Avg,PE GAAP 10Y Avg
0,2025-06-30,0.00,0.00,19.484935,0.000000,0.000000,0.000000,6204.953952,8461.081057,2025,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-75.36,-74.36,57.51,53.89,19.46,24.42,NaN,NaN,22.482021,22.251477,24.771408,25.455504
1,2025-03-31,57.51,53.89,19.372895,488.420003,1197.110200,30.911700,5611.852607,8473.831556,2025,3,NaN,236.24,216.69,76.145301,23.754879,25.898069,4.21,3.86,32.23,35.14,-75.36,-74.36,57.51,53.89,19.46,24.42,NaN,NaN,22.482021,22.251477,24.771408,25.455504
2,2024-12-31,61.21,57.69,19.810500,508.588989,1178.567800,34.471600,5881.627648,8467.894518,2024,12,5881.627648,233.36,210.17,74.832255,25.204095,27.985096,3.97,3.57,32.07,35.61,9.29,9.22,233.36,210.17,19.46,24.42,25.204095,27.985096,24.124493,22.061110,27.797731,25.272067
3,2024-09-30,59.16,51.99,18.680400,501.354000,1166.346100,30.891700,5762.484883,8451.342154,2024,9,5881.627648,226.05,200.27,73.400300,25.492081,28.773580,3.92,3.48,32.47,36.65,9.29,9.22,233.36,210.17,19.46,24.42,25.204095,27.985096,24.124493,22.061110,27.797731,25.272067
4,2024-06-28,58.36,53.12,18.281506,488.705399,1139.314000,28.457100,5460.482621,8395.388376,2024,6,5881.627648,219.14,195.93,71.975800,24.917781,27.869559,4.01,3.59,32.84,36.74,9.29,9.22,233.36,210.17,19.46,24.42,25.204095,27.985096,24.124493,22.061110,27.797731,25.272067
5,2024-03-28,54.63,47.37,18.059849,471.949169,1116.727900,27.060700,5254.354401,8388.772132,2024,3,5881.627648,215.62,191.39,70.824826,24.368585,27.453652,4.10,3.64,32.85,37.01,9.29,9.22,233.36,210.17,19.46,24.42,25.204095,27.985096,24.124493,22.061110,27.797731,25.272067
6,2023-12-29,53.90,47.79,18.378545,490.201506,1106.214712,30.133461,4769.829411,8394.128747,2023,12,4769.829411,213.53,192.43,70.303692,22.337983,24.787348,4.48,4.03,32.92,36.53,8.42,11.39,213.53,192.43,19.46,24.42,22.337983,24.787348,23.196177,21.362416,26.833653,24.485973
7,2023-09-29,52.25,47.65,17.255900,468.524082,1076.237542,27.013050,4288.054123,8380.848066,2023,9,4769.829411,210.00,184.25,69.313136,20.419305,23.273021,4.90,4.30,33.01,37.62,8.42,11.39,213.53,192.43,19.46,24.42,22.337983,24.787348,23.196177,21.362416,26.833653,24.485973
8,2023-06-30,54.84,48.58,17.130532,462.132679,1061.913100,25.961900,4450.381312,8350.369620,2023,6,4769.829411,208.10,181.01,68.714936,21.385782,24.586384,4.68,4.07,33.02,37.96,8.42,11.39,213.53,192.43,19.46,24.42,22.337983,24.787348,23.196177,21.362416,26.833653,24.485973
9,2023-03-31,52.54,48.41,17.538715,451.437952,1045.266400,25.460100,4109.312445,8357.127040,2023,3,4769.829411,200.13,175.17,68.211442,20.533216,23.458997,4.87,4.26,34.08,38.94,8.42,11.39,213.53,192.43,19.46,24.42,22.337983,24.787348,23.196177,21.362416,26.833653,24.485973


In [ ]:
sp500_eps_df.to_excel('D:\Investment\SP500_sectors\sp500_earnings\sp-500-eps-est-hist.xlsx')

# Stock PE by years by tickers

In [ ]:
stock_financial_dict = {} # the dict will store ticker as key, the string value of 'annual_df' and 'qtr_df' as nested key, actual json as value
stock_PE_annual_consolidate_df = pd.DataFrame()

ticker_symbols = [
'MNST'
,'KO'
,'KDP'
,'PEP'
 ]
# ticker_symbols = electric_utility_symbols
PE_yr_range = 10 # test for 'X' years PE 

In [ ]:

# storage process
for j, symbol in enumerate(ticker_symbols):
    print(j, symbol)
    # # COMPANY OVERVIEW
    # # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    # url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={alpha_vantage_api_key}'
    # r = requests.get(url)
    # data = r.json()


    # for key, value in data.items():
    #     if key  == 'MarketCapitalization':
    #         stock_mkt_cap = float(value)



    # STOCK SPLIT FACTOR section
    url = f'https://www.alphavantage.co/query?function=SPLITS&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'data':
            if len(value) > 0:
                stock_split_record_df = pd.DataFrame(value)
                stock_split_record_df['split_factor'] = pd.to_numeric(stock_split_record_df['split_factor'], errors='coerce') # change split_factor series to numeric data
                stock_split_record_df['effective_date'] = pd.to_datetime(stock_split_record_df['effective_date'])
            else:
                # Use pandas index assign instead of direct assign value
                stock_split_record_df = pd.DataFrame({
                'split_factor': [1],
                'effective_date': [datetime.today()]
            })



    # Monthly quote section
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'Monthly Time Series':
            Monthly_stock_df = pd.DataFrame(value)


    Monthly_stock_df = Monthly_stock_df.transpose()
    Monthly_stock_df.index = pd.to_datetime(Monthly_stock_df.index)


    filter_1 = (Monthly_stock_df.index.year.isin(range((datetime.today().year - PE_yr_range) ,datetime.today().year)))
    filter_2 = (Monthly_stock_df.index.month == 12) # month = 12 to get the year end closing price

    selected_cols = [
        '4. close'
    ]

    Monthly_stock_df = Monthly_stock_df[
        filter_1
        & filter_2
    ][selected_cols]

    # Rename columns
    Monthly_stock_df.rename(
        columns={
            '4. close': 'stock_price'
            }
        ,inplace=True
        )

    Monthly_stock_df["stock_price"] = Monthly_stock_df["stock_price"].astype(str).apply(lambda x: float(x))
    Monthly_stock_df["stock_price"] = Monthly_stock_df["stock_price"].round(2)

    # modify stock price based on stock split
    for year_i in Monthly_stock_df.index.year:
        for year_j in stock_split_record_df['effective_date'].dt.year:
            if year_i == year_j:

                # stock price to divided the split factor
                Monthly_stock_df.loc[Monthly_stock_df.index.year < year_j, 'stock_price'] /= (stock_split_record_df['split_factor'][stock_split_record_df['effective_date'].dt.year == year_j].values[0])
    


    # Earning section
    # past earnings from alpha vintage API
    url = f'https://www.alphavantage.co/query?function=EARNINGS&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'annualEarnings':

            selected_cols = [
                'fiscalDateEnding'
                ,'reportedEPS'
            ]

            annualEPS_df = pd.DataFrame(value) # tranpose the dataframe and sub select selected cols


            annualEPS_df['fiscalDateEnding'] = pd.to_datetime(annualEPS_df['fiscalDateEnding']).dt.year

            annualEPS_df = annualEPS_df[
                annualEPS_df['fiscalDateEnding'].isin(
                    range(
                        (datetime.today().year - PE_yr_range) 
                        ,datetime.today().year
                            )
                            )
                            ]

            # Convert the column to decimal type
            for col in selected_cols:
                if col in ['reportedEPS']:
                    annualEPS_df[f'{col}'] = annualEPS_df[f'{col}'].astype(str).apply(lambda x: float(x))

                else:
                    continue
            # clean annualEPS_df
            annualEPS_df = annualEPS_df.sort_values('reportedEPS', ascending=False).drop_duplicates('fiscalDateEnding')
            annualEPS_df = annualEPS_df.sort_values('fiscalDateEnding', ascending=False).reset_index(drop=True)

            # calculate PE
            annualEPS_df["PE"] = Monthly_stock_df["stock_price"].values / annualEPS_df['reportedEPS'].values


    ####################
    ### Consolidated ###
    ####################

    # annual PE df
    if stock_PE_annual_consolidate_df.empty:
        stock_PE_annual_consolidate_df['fiscalDateEnding'] = annualEPS_df['fiscalDateEnding']

    stock_PE_annual_consolidate_df[f'{symbol}_PE'] = annualEPS_df["PE"].round(2)


stock_PE_annual_consolidate_df['industry_avg_PE'] = stock_PE_annual_consolidate_df.iloc[:, 1:].sum(axis=1) / (stock_PE_annual_consolidate_df.shape[1] -1)  # Sum by row

0 MNST
1 KO
2 KDP
3 PEP


In [ ]:
stock_PE_annual_consolidate_df

,fiscalDateEnding,MNST_PE,KO_PE,KDP_PE,PEP_PE,industry_avg_PE
0,2024,43.44,26.72,23.97,24.53,29.6650
1,2023,36.46,21.91,18.61,22.29,24.8175
2,2022,22.76,25.65,21.23,26.61,24.0625
3,2021,18.68,25.41,23.04,27.79,23.7300
4,2020,17.25,28.12,22.86,26.87,23.7750
5,2019,15.35,26.23,23.73,24.76,22.5175
6,2018,13.67,22.66,8.90,19.55,16.1950
7,2017,22.77,24.02,22.11,23.20,23.0250
8,2016,17.05,21.71,20.61,29.98,22.3375
9,2015,25.08,21.48,23.18,21.91,22.9125


## Appendix

In [ ]:
# Consumer Staples

ticker_symbols = [
    'MO',
    'ADM',
    # 'BF.B', # DATA ISSUE
    'BG',
    'CPB',
    'CHD',
    'CLX',
    'KO',
    'CL',
    'CAG',
    'STZ', # !!
    'COST',
    'DG',
    'DLTR', # !!
    'EL',
    'GIS',
    'HSY',
    'HRL',
    'K',
    'KVUE',
    'KDP',
    'KMB',
    'KHC',
    'KR', # DATA ISSUE
    'LW',
    'MKC', # DATA ISSUE
    'TAP',
    'MDLZ',
    'MNST',
    'PEP',
    'PM',
    'PG',
    'SJM',
    'SYY',
    'TGT', # !!
    'TSN', # !!
    'WBA', # DATA ISSUE
    'WMT'
]

In [ ]:
# Energy

ticker_symbols = [
'APA',
 'BKR', # !! HIGH PE, LOW PROFITABILITY, HIGH FORECAST,  BUT CANNOT BEAT FORECAST, 总结与TRGP类似, 基本面更稳妥, 盈利预期高, 有待考证
 'CVX',
 'COP',
 'CTRA',
 'DVN',
 'FANG',
 'EOG',
 'EQT',
 'XOM',
 'HAL', # !!!! 与SLB类似
 'HES', # !  HIGH PE, UNDERVALUE, 市场炒作高预期, 2024表现强劲, 已经被超热了
 'KMI',
 'MRO', # PROFITABILITY, STEADY GROWTH FORECAST, AND CAN BEAT FORECAST, 总结基本面良好，与HAL类型相似, 市场炒作热度比HAL更高, 值得研究
 'TRGP', # !！ HGIH PE
 'MPC',
 'OXY',
 'OKE', # !!
 'PSX',
 'SLB', # !!! A BIT HIGH PE, GOOD ROE, MED, HIGH ROE, RELATIVE MORE DEBT THAN EQUITY, LOW PROFITABILITY, HIGH FORECAST, BUT CANNOT BEAT FORECAST, 总结基本面一般, 但是盈利预期非常高, 有待考证
 'VLO',
 'WMB'
 ]

In [ ]:
electric_utility_symbols = [
'CEPU'
,'OTTR'
,'PAM'
,'SO' ##
,'DUK'
,'NRG'
,'IDA'
,'DTE' #
,'CEG' ##
,'SRE'
,'ETR'
,'PEG'
,'MGEE'
,'LNT'
,'BKH'
]